In [1]:
import coba as cb
import torch
from pathlib import Path
from collections import defaultdict, Counter
from itertools import islice, chain, count, product, repeat
from contextlib import nullcontext

data_dir = "../data"

import coba as cb

import warnings

import time
import csv
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator
from sklearn.preprocessing import RobustScaler, QuantileTransformer, MinMaxScaler, StandardScaler, Binarizer
from sklearn.feature_selection import  mutual_info_classif, f_classif, GenericUnivariateSelect
from sklearn.model_selection import cross_validate, cross_val_predict, StratifiedKFold, GridSearchCV, LeaveOneGroupOut, StratifiedShuffleSplit
from sklearn.metrics import balanced_accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline

import torch
from parameterfree import COCOB

from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier

from concurrent.futures import ProcessPoolExecutor

import torch
import torch.utils
import torch.utils.data
import coba as cb
from typing import Tuple, Optional

try:
    torch.set_num_threads(3)
    torch.set_num_interop_threads(3)
except RuntimeError:
    pass

c0 = "#444"
c1 = "#0072B2"
c2 = "#E69F00"
c3 = "#009E73"
c4 = "#56B4E9"
c5 = "#D55E00"
c6 = "#F0E442"
c7 = "#CC79A7"
c8 = "#000000"
c9 = "#332288"

torch.set_default_device('cpu')
plt.rc('font', **{'size': 20})

df = pd.read_csv(f"{data_dir}/all_features_1h_v3.csv")

G  = df["id_participant"].to_numpy()
X1 = df[[c for c in df.columns if c.startswith("acc_")]].to_numpy()
X2 = df[[c for c in df.columns if c.startswith("acc_") or c.startswith("gps_") or c.startswith("motion_")]].to_numpy()
Y1 = df["ER_desire"].astype(float).to_numpy()
Y2 = (df["INT_availability"] == "yes").astype(float).to_numpy()

#no_na = ~(np.isnan(X2).any(axis=1) | np.isnan(Y1) | np.isnan(Y2))
no_na = ~(np.isnan(Y1) | np.isnan(Y2))
#no_na = ~(np.isnan(Y2))

G  = G [no_na]
X1 = X1[no_na]
X2 = X2[no_na]
X3 = X2.copy()
X4 = X2.copy()
Y1 = np.expand_dims(Y1[no_na],axis=1)
Y2 = np.expand_dims(Y2[no_na],axis=1)

for g in set(G):
    Y1[G == g] = Binarizer(threshold=np.mean(Y1[G == g].squeeze())).fit_transform(Y1[G == g])
    X1[G == g] = StandardScaler().fit_transform(X1[G == g])
    X2[G == g] = StandardScaler().fit_transform(X2[G == g])
    X3[G == g] = StandardScaler().fit_transform(X3[G == g])
    X4[G == g] = StandardScaler().fit_transform(X4[G == g])

X3 = np.concatenate([X3, np.expand_dims((df["Platform"] == "Android").astype(float).to_numpy(),1)[no_na]],axis=1)
X3 = np.concatenate([X3, np.expand_dims((df["Platform"] == "iPhone" ).astype(float).to_numpy(),1)[no_na]],axis=1)

X4 = np.concatenate([X4, np.expand_dims((df["Platform"] == "Android").astype(float).to_numpy(),1)[no_na]],axis=1)
X4 = np.concatenate([X4, np.expand_dims((df["Platform"] == "iPhone" ).astype(float).to_numpy(),1)[no_na]],axis=1)
X4 = np.concatenate([X4, np.expand_dims((df["tag"] == "evening" ).astype(float).to_numpy(),1)[no_na]],axis=1)
X4 = np.concatenate([X4, np.expand_dims((df["tag"] == "morning" ).astype(float).to_numpy(),1)[no_na]],axis=1)
X4 = np.concatenate([X4, np.expand_dims((df["tag"] == "afternoon" ).astype(float).to_numpy(),1)[no_na]],axis=1)

/home/mrucker/miniconda3/envs/diss/lib/python3.10/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/mrucker/miniconda3/envs/diss/lib/python3.10/site-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/home/mrucker/miniconda3/envs/diss/lib/python3.10/site-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/home/mrucker/miniconda3/envs/diss/lib/python3.10/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/mrucker/miniconda3/envs/diss/lib/python3.10/site-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/home/mrucker

In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr):

        by_class  = {y: self.train_X[(self.train_Y==y)].tolist() for y in set(self.train_Y.tolist())}
        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        for _ in range(sr):
            anchor, positive, negative = [], [], []
            for x,y in zip(self.train_X.tolist(),self.train_Y.tolist()):
                anchor.append(x)
                positive.append(choose_unique(by_class[y],x))
                negative.append(choose_n(by_class[1-y],neg))

            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s1, s2, s3, ssl_samps, ssl_neg, ssl_dropn, ssl_tau, ws_steps, pers_lrn_cnt, pers_mem_cnt, pers_mem_rpt, pers_mem_rcl):

        self.s1  = s1  #ssl + sl
        self.s2  = s2  #sl
        self.s3  = s3  #sl + online

        self.ssl_samps = ssl_samps
        self.ssl_neg   = ssl_neg
        self.ssl_tau   = ssl_tau
        self.ssl_dropn = ssl_dropn
        
        self.ws_steps = ws_steps

        self.pers_lrn_cnt = pers_lrn_cnt 
        self.pers_mem_cnt = pers_mem_cnt
        self.pers_mem_rpt = pers_mem_rpt
        self.pers_mem_rcl = pers_mem_rcl
        
        self.params = {'s1': s1, 's2':s2, 's3':s3, 'ssl': (ssl_samps,ssl_neg,ssl_dropn,ssl_tau), 'ws': ws_steps, 'pers': (pers_lrn_cnt,pers_mem_cnt,pers_mem_rpt,pers_mem_rcl)}

    def evaluate(self, env, lrn):
        from sklearn.metrics import roc_auc_score

        torch.set_num_threads(1)
        torch.manual_seed(1)

        s1 = FeedForward(self.s1[:len(self.s1)-self.ssl_dropn])
        sb = FeedForward(self.s1[len(self.s1)-self.ssl_dropn:])
        s2 = FeedForward(self.s2)
        s3 = FeedForward(self.s3)

        s1.train()
        sb.train()
        s2.train()
        s3.train()

        s1opt = COCOB(s1.parameters()) if list(s1.parameters()) else None
        sbopt = COCOB(sb.parameters()) if list(sb.parameters()) else None
        s2opt = COCOB(s2.parameters()) if list(s2.parameters()) else None
        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        if self.ssl_samps:

            if self.ssl_neg == 0: raise Exception("neg can't be 0")
            if self.ssl_tau == 0: raise Exception("Tau can't be 0")

            torch.manual_seed(1)
            
            for A, P, N in env.ssl(self.ssl_neg,self.ssl_samps):

                torch_dataset = torch.utils.data.TensorDataset(A,P,N)
                torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=64,drop_last=True,shuffle=True)

                for _A, _P, _N in torch_loader:
                    #https://arxiv.org/pdf/2002.05709
                    _A = sb(s1(_A))
                    _P = sb(s1(_P))
                    _N = sb(s1(_N))

                    p = torch.einsum("bi,bi->b",_A,_P)
                    n = torch.einsum("bi,bji->bj",_A,_N)

                    p /= (torch.linalg.norm(_A,dim=1)*torch.linalg.norm(_P,dim=1))
                    n /= (torch.linalg.norm(_A,dim=1).unsqueeze(1)*torch.linalg.norm(_N,dim=2))

                    p = torch.exp(p/self.ssl_tau)
                    n = torch.exp(n/self.ssl_tau)

                    if s1opt: s1opt.zero_grad()
                    if sbopt: sbopt.zero_grad()
                    (-torch.log(p/(p+n.sum(dim=1)))).mean().backward()
                    if s1opt: s1opt.step()
                    if sbopt: sbopt.step()

        if self.ws_steps:

            if s1opt: s1opt.zero_grad()
            if s2opt: s2opt.zero_grad()
            if s3opt: s3opt.zero_grad()

            X, Y = env.train()

            loss = torch.nn.BCEWithLogitsLoss()

            torch.manual_seed(1)

            torch_dataset = torch.utils.data.TensorDataset(X,Y)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            for _ in range(self.ws_steps):
                for _X,_y in torch_loader:

                    if s1opt: s1opt.zero_grad()
                    if s2opt: s2opt.zero_grad()
                    if s3opt: s3opt.zero_grad()
                    loss(s3(s2(s1(_X))),_y).backward()
                    if s3opt: s3opt.step()
                    if s2opt: s2opt.step()
                    if s1opt: s1opt.step()

        s1.eval()
        s2.eval()
        s3.eval()

        torch.manual_seed(1)

        loss = torch.nn.BCEWithLogitsLoss()

        X, Y = env.test()

        rng = cb.CobaRandom(1)
        while len(set(Y[35:].squeeze().tolist())) == 1:
            ind = rng.shuffle(range(len(Y)))
            X = X[ind,:]
            Y = Y[ind]

        if s1opt: s1opt.zero_grad()
        if s2opt: s2opt.zero_grad()

        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        lrnx = []
        lrny = []
        mems = []

        with torch.no_grad():
            yield {'auc': roc_auc_score(Y[35:],torch.sigmoid(s3(s2(s1(X[35:,:])))))}
        
        for i in range(35):

            x,y = X[i,:],Y[i]

            if self.pers_lrn_cnt:
                lrnx.append(x)
                lrny.append(y)

            if self.pers_mem_cnt: 
                mems.append([x,y,self.pers_mem_rpt])

            if len(mems) > self.pers_mem_cnt:
                rng.shuffle(mems, inplace=True)
                for j in reversed(range(self.pers_mem_rcl)):
                    x,y,n = mems[j]
                    lrnx.append(x)
                    lrny.append(y)
                    if n == 1: mems.pop(j)
                    else: mems[j] = [x,y,n-1]

            if len(lrnx) >= self.pers_lrn_cnt:
                x = torch.stack(lrnx[:self.pers_lrn_cnt])
                y = torch.stack(lrny[:self.pers_lrn_cnt])
                if s3opt: s3opt.zero_grad()
                loss(s3(s2(s1(x))),y).backward()
                if s3opt: s3opt.step()
                lrnx = lrnx[self.pers_lrn_cnt:]
                lrny = lrny[self.pers_lrn_cnt:]

            with torch.no_grad():
                yield {'auc': roc_auc_score(Y[35:],torch.sigmoid(s3(s2(s1(X[35:,:])))))}

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(5):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if len(set(tst_y.squeeze().tolist())) == 1:
                all_equal.add(g)
            else:
                envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))
    
    if len(all_equal) > 0:
        print(f"All Equal, no environment added for {sorted(all_equal)}") 

    return envs

r = lambda w:  ['l', 'r', w, 'l', 'r', w]

lrns = [ FeedForward([]) ]

vals = lambda x: [
    MyEvaluator((), (len(x),30,'l','r'), (30,1), 0, 0, 0, 0, 3, 3, 20, 2, 2),
    MyEvaluator((), (len(x),30,'l','r'), (30,1), 0, 0, 0, 0, 3, 2, 20, 2, 2),
    MyEvaluator((), (len(x),30,'l','r'), (30,1), 0, 0, 0, 0, 3, 2, 20, 1, 1),
    MyEvaluator((), (len(x),30,'l','r'), (30,1), 0, 0, 0, 0, 3, 2,  0, 0, 0),
    MyEvaluator((), (len(x),30,'l','r'), (30,1), 0, 0, 0, 0, 3, 1,  0, 0, 0),
    MyEvaluator((), (len(x),30,'l','r'), (30,1), 0, 0, 0, 0, 3, 1, 10, 1, 1),
    MyEvaluator((), (len(x),30,'l','r'), (30,1), 0, 0, 0, 0, 3, 1, 20, 2, 2),
]

envs = make_envs(X3,Y2,G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/5/out5_1.log',processes=35,quiet=True)


All Equal, no environment added for [82, 87, 89, 99, 103, 111, 117, 146, 196, 227, 246, 311, 357, 377, 390, 451, 455]
{'Learners': 1, 'Environments': 910, 'Interactions': 229320}


In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr):

        by_class  = {y: self.train_X[(self.train_Y==y)].tolist() for y in set(self.train_Y.tolist())}
        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        for _ in range(sr):
            anchor, positive, negative = [], [], []
            for x,y in zip(self.train_X.tolist(),self.train_Y.tolist()):
                anchor.append(x)
                positive.append(choose_unique(by_class[y],x))
                negative.append(choose_n(by_class[1-y],neg))

            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s0, s1, s2, s3, dae_steps, dae_dropn, ssl_samps, ssl_neg, ssl_dropn, ssl_tau, ws_steps, pers_lrn_cnt, pers_mem_cnt, pers_mem_rpt, pers_mem_rcl):

        self.s0  = s0  #dae + sl
        self.s1  = s1  #ssl + sl
        self.s2  = s2  #sl
        self.s3  = s3  #sl + online

        self.dae_steps = dae_steps
        self.dae_dropn = dae_dropn

        self.ssl_samps = ssl_samps
        self.ssl_neg   = ssl_neg
        self.ssl_tau   = ssl_tau
        self.ssl_dropn = ssl_dropn

        self.ws_steps = ws_steps

        self.pers_lrn_cnt = pers_lrn_cnt
        self.pers_mem_cnt = pers_mem_cnt
        self.pers_mem_rpt = pers_mem_rpt
        self.pers_mem_rcl = pers_mem_rcl

        self.params = {'s0', s0, 's1': s1, 's2':s2, 's3':s3, 'dae': (dae_steps,dae_dropn), 'ssl': (ssl_samps,ssl_neg,ssl_dropn,ssl_tau), 'ws': ws_steps, 'pers': (pers_lrn_cnt,pers_mem_cnt,pers_mem_rpt,pers_mem_rcl)}

    def evaluate(self, env, lrn):
        from sklearn.metrics import roc_auc_score

        torch.set_num_threads(1)
        torch.manual_seed(1)

        s0 = FeedForward(self.s0)
        s1 = FeedForward(self.s1)
        s2 = FeedForward(self.s2)
        s3 = FeedForward(self.s3)

        sa = torch.nn.Sequential(*list(s0.children())[len(self.s0)-self.dae_dropn:])
        s0 = torch.nn.Sequential(*list(s0.children())[:len(self.s0)-self.dae_dropn])

        sb = torch.nn.Sequential(*list(s1.children())[len(self.s1)-self.ssl_dropn:])
        s1 = torch.nn.Sequential(*list(s1.children())[:len(self.s1)-self.ssl_dropn])

        s0.train()
        sa.train()
        s1.train()
        sb.train()
        s2.train()
        s3.train()

        s0opt = COCOB(s0.parameters()) if list(s0.parameters()) else None
        saopt = COCOB(sa.parameters()) if list(sa.parameters()) else None
        s1opt = COCOB(s1.parameters()) if list(s1.parameters()) else None
        sbopt = COCOB(sb.parameters()) if list(sb.parameters()) else None
        s2opt = COCOB(s2.parameters()) if list(s2.parameters()) else None
        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        if self.dae_steps:

            torch.manual_seed(1)
            X, Y = env.train()
            torch_dataset = torch.utils.data.TensorDataset(X)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.L1Loss(reduction='none')
            for _ in range(self.dae_steps):
                for (_X,) in torch_loader:
                    if s0opt: s0opt.zero_grad()
                    if saopt: saopt.zero_grad()
                    loss(sa(s0(_X.nan_to_num())),_X).nanmean().backward()
                    if s0opt: s0opt.step()
                    if saopt: saopt.step()

        if self.ssl_samps:

            if self.ssl_neg == 0: raise Exception("neg can't be 0")
            if self.ssl_tau == 0: raise Exception("Tau can't be 0")

            torch.manual_seed(1)

            for A, P, N in env.ssl(self.ssl_neg,self.ssl_samps):

                torch_dataset = torch.utils.data.TensorDataset(A,P,N)
                torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=64,drop_last=True,shuffle=True)

                for _A, _P, _N in torch_loader:
                    #https://arxiv.org/pdf/2002.05709
                    _A = sb(s1(s0(_A.nan_to_num())))
                    _P = sb(s1(s0(_P.nan_to_num())))
                    _N = sb(s1(s0(_N.nan_to_num())))

                    p = torch.einsum("bi,bi->b",_A,_P)
                    n = torch.einsum("bi,bji->bj",_A,_N)

                    p /= (torch.linalg.norm(_A,dim=1)*torch.linalg.norm(_P,dim=1))
                    n /= (torch.linalg.norm(_A,dim=1).unsqueeze(1)*torch.linalg.norm(_N,dim=2))

                    p = torch.exp(p/self.ssl_tau)
                    n = torch.exp(n/self.ssl_tau)

                    if s1opt: s1opt.zero_grad()
                    if sbopt: sbopt.zero_grad()
                    (-torch.log(p/(p+n.sum(dim=1)))).mean().backward()
                    if s1opt: s1opt.step()
                    if sbopt: sbopt.step()

        if self.ws_steps:

            if s0opt: s0opt.zero_grad()
            if s1opt: s1opt.zero_grad()
            if s2opt: s2opt.zero_grad()
            if s3opt: s3opt.zero_grad()

            torch.manual_seed(1)
            X, Y = env.train()
            torch_dataset = torch.utils.data.TensorDataset(X,Y)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.BCEWithLogitsLoss()
            for _ in range(self.ws_steps):
                for _X,_y in torch_loader:

                    if s0opt: s0opt.zero_grad()
                    if s1opt: s1opt.zero_grad()
                    if s2opt: s2opt.zero_grad()
                    if s3opt: s3opt.zero_grad()
                    loss(s3(s2(s1(s0(_X.nan_to_num())))),_y).backward()
                    if s3opt: s3opt.step()
                    if s2opt: s2opt.step()
                    if s1opt: s1opt.step()
                    if s0opt: s0opt.step()

        s0.eval()
        s1.eval()
        s2.eval()
        s3.eval()

        torch.manual_seed(1)

        X, Y = env.test()

        rng = cb.CobaRandom(1)
        while len(set(Y[35:].squeeze().tolist())) == 1:
            ind = rng.shuffle(range(len(Y)))
            X = X[ind,:]
            Y = Y[ind]

        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        lrnx = []
        lrny = []
        mems = []

        with torch.no_grad():
            yield {'auc': roc_auc_score(Y[35:],torch.sigmoid(s3(s2(s1(s0(X[35:,:].nan_to_num()))))))}

        loss = torch.nn.BCEWithLogitsLoss()
        for i in range(35):

            x,y = X[i,:],Y[i]

            if self.pers_lrn_cnt:
                lrnx.append(x)
                lrny.append(y)

            if self.pers_mem_cnt: 
                mems.append([x,y,self.pers_mem_rpt])

            if len(mems) > self.pers_mem_cnt:
                rng.shuffle(mems, inplace=True)
                for j in reversed(range(self.pers_mem_rcl)):
                    x,y,n = mems[j]
                    lrnx.append(x)
                    lrny.append(y)
                    if n == 1: mems.pop(j)
                    else: mems[j] = [x,y,n-1]

            if len(lrnx) >= self.pers_lrn_cnt:
                x = torch.stack(lrnx[:self.pers_lrn_cnt])
                y = torch.stack(lrny[:self.pers_lrn_cnt])
                if s3opt: s3opt.zero_grad()
                loss(s3(s2(s1(s0(x.nan_to_num())))),y).backward()
                if s3opt: s3opt.step()
                lrnx = lrnx[self.pers_lrn_cnt:]
                lrny = lrny[self.pers_lrn_cnt:]

            with torch.no_grad():
                yield {'auc': roc_auc_score(Y[35:],torch.sigmoid(s3(s2(s1(s0(X[35:,:].nan_to_num()))))))}

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(5):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if len(set(tst_y.squeeze().tolist())) == 1:
                all_equal.add(g)
            else:
                envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))

    if len(all_equal) > 0:
        print(f"All Equal, no environment added for {sorted(all_equal)}")

    return envs

r = lambda w:  ['l', 'r', w, 'l', 'r', w]

lrns = [ FeedForward([]) ]

vals = lambda x: [
    MyEvaluator((), (), (len(x),30,'l','r'), (30,1), 0, 0, 0, 0, 0, 0, 3, 3, 20, 2, 2),
    MyEvaluator((len(x),.2,90,'l','r',len(x)), (), (90,30,'l','r'), (30,1), 16, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2),
]

envs = make_envs(X3,Y2,G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/5/out5_2.log',processes=35,quiet=True)


All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 451, 455]
{'Learners': 1, 'Environments': 661, 'Interactions': 46332}


In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr):

        by_class  = {y: self.train_X[(self.train_Y==y)].tolist() for y in set(self.train_Y.tolist())}
        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        for _ in range(sr):
            anchor, positive, negative = [], [], []
            for x,y in zip(self.train_X.tolist(),self.train_Y.tolist()):
                anchor.append(x)
                positive.append(choose_unique(by_class[y],x))
                negative.append(choose_n(by_class[1-y],neg))

            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s0, s1, s2, s3, dae_steps, dae_dropn, ssl_samps, ssl_neg, ssl_dropn, ssl_tau, ws_steps, pers_lrn_cnt, pers_mem_cnt, pers_mem_rpt, pers_mem_rcl, v):

        self.s0  = s0  #dae + sl
        self.s1  = s1  #ssl + sl
        self.s2  = s2  #sl
        self.s3  = s3  #sl + online

        self.dae_steps = dae_steps
        self.dae_dropn = dae_dropn

        self.ssl_samps = ssl_samps
        self.ssl_neg   = ssl_neg
        self.ssl_tau   = ssl_tau
        self.ssl_dropn = ssl_dropn

        self.ws_steps = ws_steps

        self.pers_lrn_cnt = pers_lrn_cnt
        self.pers_mem_cnt = pers_mem_cnt
        self.pers_mem_rpt = pers_mem_rpt
        self.pers_mem_rcl = pers_mem_rcl

        self.v = v

        self.params = {'s0': s0, 's1': s1, 's2':s2, 's3':s3, 'dae': (dae_steps,dae_dropn), 'ssl': (ssl_samps,ssl_neg,ssl_dropn,ssl_tau), 'ws': ws_steps, 'pers': (pers_lrn_cnt,pers_mem_cnt,pers_mem_rpt,pers_mem_rcl)}

    def evaluate(self, env, lrn):
        from sklearn.metrics import roc_auc_score

        torch.set_num_threads(1)
        torch.manual_seed(1)

        s0 = FeedForward(self.s0)
        s1 = FeedForward(self.s1)
        s2 = FeedForward(self.s2)
        s3 = FeedForward(self.s3)

        sa = torch.nn.Sequential(*list(s0.children())[len(self.s0)-self.dae_dropn:])
        s0 = torch.nn.Sequential(*list(s0.children())[:len(self.s0)-self.dae_dropn])

        sb = torch.nn.Sequential(*list(s1.children())[len(self.s1)-self.ssl_dropn:])
        s1 = torch.nn.Sequential(*list(s1.children())[:len(self.s1)-self.ssl_dropn])

        s0.train()
        sa.train()
        s1.train()
        sb.train()
        s2.train()
        s3.train()

        s0opt = COCOB(s0.parameters()) if list(s0.parameters()) else None
        saopt = COCOB(sa.parameters()) if list(sa.parameters()) else None
        s1opt = COCOB(s1.parameters()) if list(s1.parameters()) else None
        sbopt = COCOB(sb.parameters()) if list(sb.parameters()) else None
        s2opt = COCOB(s2.parameters()) if list(s2.parameters()) else None
        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        if self.dae_steps:

            torch.manual_seed(1)
            X, Y = env.train()
            torch_dataset = torch.utils.data.TensorDataset(X)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.L1Loss(reduction='none')
            for _ in range(self.dae_steps):
                for (_X,) in torch_loader:
                    if s0opt: s0opt.zero_grad()
                    if saopt: saopt.zero_grad()
                    loss(sa(s0(_X.nan_to_num())),_X).nanmean().backward()
                    if s0opt: s0opt.step()
                    if saopt: saopt.step()

        if self.ssl_samps:

            if self.ssl_neg == 0: raise Exception("neg can't be 0")
            if self.ssl_tau == 0: raise Exception("Tau can't be 0")

            torch.manual_seed(1)

            for A, P, N in env.ssl(self.ssl_neg,self.ssl_samps):

                torch_dataset = torch.utils.data.TensorDataset(A,P,N)
                torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=64,drop_last=True,shuffle=True)

                for _A, _P, _N in torch_loader:
                    #https://arxiv.org/pdf/2002.05709
                    _A = sb(s1(s0(_A.nan_to_num())))
                    _P = sb(s1(s0(_P.nan_to_num())))
                    _N = sb(s1(s0(_N.nan_to_num())))

                    p = torch.einsum("bi,bi->b",_A,_P)
                    n = torch.einsum("bi,bji->bj",_A,_N)

                    p /= (torch.linalg.norm(_A,dim=1)*torch.linalg.norm(_P,dim=1))
                    n /= (torch.linalg.norm(_A,dim=1).unsqueeze(1)*torch.linalg.norm(_N,dim=2))

                    p = torch.exp(p/self.ssl_tau)
                    n = torch.exp(n/self.ssl_tau)

                    if s1opt: s1opt.zero_grad()
                    if sbopt: sbopt.zero_grad()
                    (-torch.log(p/(p+n.sum(dim=1)))).mean().backward()
                    if s1opt: s1opt.step()
                    if sbopt: sbopt.step()

        if self.ws_steps:

            if s0opt and self.v: s0opt.zero_grad()
            if s1opt: s1opt.zero_grad()
            if s2opt: s2opt.zero_grad()
            if s3opt: s3opt.zero_grad()

            torch.manual_seed(1)
            X, Y = env.train()
            torch_dataset = torch.utils.data.TensorDataset(X,Y)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.BCEWithLogitsLoss()
            for _ in range(self.ws_steps):
                for _X,_y in torch_loader:

                    if s0opt and self.v: s0opt.zero_grad()
                    if s1opt: s1opt.zero_grad()
                    if s2opt: s2opt.zero_grad()
                    if s3opt: s3opt.zero_grad()
                    loss(s3(s2(s1(s0(_X.nan_to_num())))),_y).backward()
                    if s3opt: s3opt.step()
                    if s2opt: s2opt.step()
                    if s1opt: s1opt.step()
                    if s0opt and self.v: s0opt.step()

        s0.eval()
        s1.eval()
        s2.eval()
        s3.eval()

        torch.manual_seed(1)

        X, Y = env.test()

        rng = cb.CobaRandom(1)
        while len(set(Y[35:].squeeze().tolist())) == 1:
            ind = rng.shuffle(range(len(Y)))
            X = X[ind,:]
            Y = Y[ind]

        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        lrnx = []
        lrny = []
        mems = []

        with torch.no_grad():
            yield {'auc': roc_auc_score(Y[35:],torch.sigmoid(s3(s2(s1(s0(X[35:,:].nan_to_num()))))))}

        loss = torch.nn.BCEWithLogitsLoss()
        for i in range(35):

            x,y = X[i,:],Y[i]

            if self.pers_lrn_cnt:
                lrnx.append(x)
                lrny.append(y)

            if self.pers_mem_cnt: 
                mems.append([x,y,self.pers_mem_rpt])

            if len(mems) > self.pers_mem_cnt:
                rng.shuffle(mems, inplace=True)
                for j in reversed(range(self.pers_mem_rcl)):
                    x,y,n = mems[j]
                    lrnx.append(x)
                    lrny.append(y)
                    if n == 1: mems.pop(j)
                    else: mems[j] = [x,y,n-1]

            if len(lrnx) >= self.pers_lrn_cnt:
                x = torch.stack(lrnx[:self.pers_lrn_cnt])
                y = torch.stack(lrny[:self.pers_lrn_cnt])
                if s3opt: s3opt.zero_grad()
                loss(s3(s2(s1(s0(x.nan_to_num())))),y).backward()
                if s3opt: s3opt.step()
                lrnx = lrnx[self.pers_lrn_cnt:]
                lrny = lrny[self.pers_lrn_cnt:]

            with torch.no_grad():
                yield {'auc': roc_auc_score(Y[35:],torch.sigmoid(s3(s2(s1(s0(X[35:,:].nan_to_num()))))))}

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if len(set(tst_y.squeeze().tolist())) == 1:
                all_equal.add(g)
            else:
                envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))

    if len(all_equal) > 0:
        print(f"All Equal, no environment added for {sorted(all_equal)}")

    return envs

r = lambda w:  ['l', 'r', w, 'l', 'r', w]

lrns = [ FeedForward([]) ]

vals = lambda x: [
    MyEvaluator((len(x),.1,90,'l','r',len(x)), (), (90,30,'l','r'), (30,1), 8, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.2,90,'l','r',len(x)), (), (90,30,'l','r'), (30,1), 8, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.3,90,'l','r',len(x)), (), (90,30,'l','r'), (30,1), 8, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.2,90,'l','r',len(x)), (), (90,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.2,90,'l','r',len(x)), (), (90,30,'l','r'), (30,1), 4, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.2,90,'l','r',len(x)), (), (90,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, False),
    MyEvaluator((len(x),.2,90,'l','r',len(x)), (), (90,30,'l','r'), (30,1), 0, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),   90,'l','r',len(x)), (), (90,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
]

envs = make_envs(X3,Y2,G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/5/out5_3.log',processes=35,quiet=True)


All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 451, 455]
{'Learners': 1, 'Environments': 228, 'Interactions': 65664}


In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr):

        by_class  = {y: self.train_X[(self.train_Y==y)].tolist() for y in set(self.train_Y.tolist())}
        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        for _ in range(sr):
            anchor, positive, negative = [], [], []
            for x,y in zip(self.train_X.tolist(),self.train_Y.tolist()):
                anchor.append(x)
                positive.append(choose_unique(by_class[y],x))
                negative.append(choose_n(by_class[1-y],neg))

            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s0, s1, s2, s3, dae_steps, dae_dropn, ssl_samps, ssl_neg, ssl_dropn, ssl_tau, ws_steps, pers_lrn_cnt, pers_mem_cnt, pers_mem_rpt, pers_mem_rcl, v):

        self.s0  = s0  #dae + sl
        self.s1  = s1  #ssl + sl
        self.s2  = s2  #sl
        self.s3  = s3  #sl + online

        self.dae_steps = dae_steps
        self.dae_dropn = dae_dropn

        self.ssl_samps = ssl_samps
        self.ssl_neg   = ssl_neg
        self.ssl_tau   = ssl_tau
        self.ssl_dropn = ssl_dropn

        self.ws_steps = ws_steps

        self.pers_lrn_cnt = pers_lrn_cnt
        self.pers_mem_cnt = pers_mem_cnt
        self.pers_mem_rpt = pers_mem_rpt
        self.pers_mem_rcl = pers_mem_rcl

        self.v = v

        self.params = {'s0': s0, 's1': s1, 's2':s2, 's3':s3, 'dae': (dae_steps,dae_dropn), 'ssl': (ssl_samps,ssl_neg,ssl_dropn,ssl_tau), 'ws': ws_steps, 'pers': (pers_lrn_cnt,pers_mem_cnt,pers_mem_rpt,pers_mem_rcl)}

    def evaluate(self, env, lrn):
        from sklearn.metrics import roc_auc_score

        torch.set_num_threads(1)
        torch.manual_seed(1)

        s0 = FeedForward(self.s0)
        s1 = FeedForward(self.s1)
        s2 = FeedForward(self.s2)
        s3 = FeedForward(self.s3)

        sa = torch.nn.Sequential(*list(s0.children())[len(self.s0)-self.dae_dropn:])
        s0 = torch.nn.Sequential(*list(s0.children())[:len(self.s0)-self.dae_dropn])

        sb = torch.nn.Sequential(*list(s1.children())[len(self.s1)-self.ssl_dropn:])
        s1 = torch.nn.Sequential(*list(s1.children())[:len(self.s1)-self.ssl_dropn])

        s0.train()
        sa.train()
        s1.train()
        sb.train()
        s2.train()
        s3.train()

        s0opt = COCOB(s0.parameters()) if list(s0.parameters()) else None
        saopt = COCOB(sa.parameters()) if list(sa.parameters()) else None
        s1opt = COCOB(s1.parameters()) if list(s1.parameters()) else None
        sbopt = COCOB(sb.parameters()) if list(sb.parameters()) else None
        s2opt = COCOB(s2.parameters()) if list(s2.parameters()) else None
        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        if self.dae_steps:

            torch.manual_seed(1)
            X, Y = env.train()
            torch_dataset = torch.utils.data.TensorDataset(X)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.L1Loss(reduction='none')
            for _ in range(self.dae_steps):
                for (_X,) in torch_loader:
                    if s0opt: s0opt.zero_grad()
                    if saopt: saopt.zero_grad()
                    loss(sa(s0(_X.nan_to_num())),_X).nanmean().backward()
                    if s0opt: s0opt.step()
                    if saopt: saopt.step()

        if self.ssl_samps:

            if self.ssl_neg == 0: raise Exception("neg can't be 0")
            if self.ssl_tau == 0: raise Exception("Tau can't be 0")

            torch.manual_seed(1)

            for A, P, N in env.ssl(self.ssl_neg,self.ssl_samps):

                torch_dataset = torch.utils.data.TensorDataset(A,P,N)
                torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=64,drop_last=True,shuffle=True)

                for _A, _P, _N in torch_loader:
                    #https://arxiv.org/pdf/2002.05709
                    _A = sb(s1(s0(_A.nan_to_num())))
                    _P = sb(s1(s0(_P.nan_to_num())))
                    _N = sb(s1(s0(_N.nan_to_num())))

                    p = torch.einsum("bi,bi->b",_A,_P)
                    n = torch.einsum("bi,bji->bj",_A,_N)

                    p /= (torch.linalg.norm(_A,dim=1)*torch.linalg.norm(_P,dim=1))
                    n /= (torch.linalg.norm(_A,dim=1).unsqueeze(1)*torch.linalg.norm(_N,dim=2))

                    p = torch.exp(p/self.ssl_tau)
                    n = torch.exp(n/self.ssl_tau)

                    if s1opt: s1opt.zero_grad()
                    if sbopt: sbopt.zero_grad()
                    (-torch.log(p/(p+n.sum(dim=1)))).mean().backward()
                    if s1opt: s1opt.step()
                    if sbopt: sbopt.step()

        if self.ws_steps:

            if s0opt and self.v: s0opt.zero_grad()
            if s1opt: s1opt.zero_grad()
            if s2opt: s2opt.zero_grad()
            if s3opt: s3opt.zero_grad()

            torch.manual_seed(1)
            X, Y = env.train()
            torch_dataset = torch.utils.data.TensorDataset(X,Y)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.BCEWithLogitsLoss()
            for _ in range(self.ws_steps):
                for _X,_y in torch_loader:

                    if s0opt and self.v: s0opt.zero_grad()
                    if s1opt: s1opt.zero_grad()
                    if s2opt: s2opt.zero_grad()
                    if s3opt: s3opt.zero_grad()
                    loss(s3(s2(s1(s0(_X.nan_to_num())))),_y).backward()
                    if s3opt: s3opt.step()
                    if s2opt: s2opt.step()
                    if s1opt: s1opt.step()
                    if s0opt and self.v: s0opt.step()

        s0.eval()
        s1.eval()
        s2.eval()
        s3.eval()

        torch.manual_seed(1)

        X, Y = env.test()

        rng = cb.CobaRandom(1)
        while len(set(Y[35:].squeeze().tolist())) == 1:
            ind = rng.shuffle(range(len(Y)))
            X = X[ind,:]
            Y = Y[ind]

        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        lrnx = []
        lrny = []
        mems = []

        with torch.no_grad():
            yield {'auc': roc_auc_score(Y[35:],torch.sigmoid(s3(s2(s1(s0(X[35:,:].nan_to_num()))))))}

        loss = torch.nn.BCEWithLogitsLoss()
        for i in range(35):

            x,y = X[i,:],Y[i]

            if self.pers_lrn_cnt:
                lrnx.append(x)
                lrny.append(y)

            if self.pers_mem_cnt: 
                mems.append([x,y,self.pers_mem_rpt])

            if len(mems) > self.pers_mem_cnt:
                rng.shuffle(mems, inplace=True)
                for j in reversed(range(self.pers_mem_rcl)):
                    x,y,n = mems[j]
                    lrnx.append(x)
                    lrny.append(y)
                    if n == 1: mems.pop(j)
                    else: mems[j] = [x,y,n-1]

            if len(lrnx) >= self.pers_lrn_cnt:
                x = torch.stack(lrnx[:self.pers_lrn_cnt])
                y = torch.stack(lrny[:self.pers_lrn_cnt])
                if s3opt: s3opt.zero_grad()
                loss(s3(s2(s1(s0(x.nan_to_num())))),y).backward()
                if s3opt: s3opt.step()
                lrnx = lrnx[self.pers_lrn_cnt:]
                lrny = lrny[self.pers_lrn_cnt:]

            with torch.no_grad():
                yield {'auc': roc_auc_score(Y[35:],torch.sigmoid(s3(s2(s1(s0(X[35:,:].nan_to_num()))))))}

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if len(set(tst_y.squeeze().tolist())) == 1:
                all_equal.add(g)
            else:
                envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))

    if len(all_equal) > 0:
        print(f"All Equal, no environment added for {sorted(all_equal)}")

    return envs

r = lambda w:  ['l', 'r', w, 'l', 'r', w]

lrns = [ FeedForward([]) ]

vals = lambda x: [
    MyEvaluator((len(x),.1,90 ,'l','r',len(x)), (), (90 ,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90 ,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.3,90 ,'l','r',len(x)), (), (90 ,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x)   ,45 ,'l','r',len(x)), (), (45 ,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.1,45 ,'l','r',len(x)), (), (45 ,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.2,45 ,'l','r',len(x)), (), (45 ,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.3,45 ,'l','r',len(x)), (), (45 ,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x)   ,30 ,'l','r',len(x)), (), (30 ,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.1,30 ,'l','r',len(x)), (), (30 ,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.2,30 ,'l','r',len(x)), (), (30 ,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.3,30 ,'l','r',len(x)), (), (30 ,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.2,75 ,'l','r',len(x)), (), (75 ,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.2,120,'l','r',len(x)), (), (120,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.2,90 ,'l','r',90,'l','r',len(x)), (), (90,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.2,90 ,'l','r',30,'l','r',len(x)), (), (30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (), (90,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
    MyEvaluator((len(x),.2,90 ,.2,'r',90,'l','r',len(x)), (), (90,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, True),
]

envs = make_envs(X3,Y2,G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/5/out5_4.log',processes=35,quiet=True)


All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 451, 455]
{'Learners': 1, 'Environments': 228, 'Interactions': 139536}


In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr):

        by_class  = {y: self.train_X[(self.train_Y==y)].tolist() for y in set(self.train_Y.tolist())}
        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        for _ in range(sr):
            anchor, positive, negative = [], [], []
            for x,y in zip(self.train_X.tolist(),self.train_Y.tolist()):
                anchor.append(x)
                positive.append(choose_unique(by_class[y],x))
                negative.append(choose_n(by_class[1-y],neg))

            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s0, s1, s2, s3, dae_steps, dae_dropn, ssl_samps, ssl_neg, ssl_dropn, ssl_tau, ws_steps, pers_lrn_cnt, pers_mem_cnt, pers_mem_rpt, pers_mem_rcl, y):

        self.s0  = s0  #dae + sl
        self.s1  = s1  #ssl + sl
        self.s2  = s2  #sl
        self.s3  = s3  #sl + pers

        self.dae_steps = dae_steps
        self.dae_dropn = dae_dropn

        self.ssl_samps = ssl_samps
        self.ssl_neg   = ssl_neg
        self.ssl_tau   = ssl_tau
        self.ssl_dropn = ssl_dropn

        self.ws_steps = ws_steps

        self.pers_lrn_cnt = pers_lrn_cnt
        self.pers_mem_cnt = pers_mem_cnt
        self.pers_mem_rpt = pers_mem_rpt
        self.pers_mem_rcl = pers_mem_rcl

        self.y = y

        self.params = {'s0': s0, 's1': s1, 's2':s2, 's3':s3, 'dae': (dae_steps,dae_dropn), 'ssl': (ssl_samps,ssl_neg,ssl_dropn,ssl_tau), 'ws': ws_steps, 'pers': (pers_lrn_cnt,pers_mem_cnt,pers_mem_rpt,pers_mem_rcl), 'y': y}

    def evaluate(self, env, lrn):
        from sklearn.metrics import roc_auc_score

        torch.set_num_threads(1)
        torch.manual_seed(1)

        s0 = FeedForward(self.s0)
        s1 = FeedForward(self.s1)
        s2 = FeedForward(self.s2)
        s3 = FeedForward(self.s3)

        sa = torch.nn.Sequential(*list(s0.children())[len(self.s0)-self.dae_dropn:])
        s0 = torch.nn.Sequential(*list(s0.children())[:len(self.s0)-self.dae_dropn])

        sb = torch.nn.Sequential(*list(s1.children())[len(self.s1)-self.ssl_dropn:])
        s1 = torch.nn.Sequential(*list(s1.children())[:len(self.s1)-self.ssl_dropn])

        s0.train()
        sa.train()
        s1.train()
        sb.train()
        s2.train()
        s3.train()

        s0opt = COCOB(s0.parameters()) if list(s0.parameters()) else None
        saopt = COCOB(sa.parameters()) if list(sa.parameters()) else None
        s1opt = COCOB(s1.parameters()) if list(s1.parameters()) else None
        sbopt = COCOB(sb.parameters()) if list(sb.parameters()) else None
        s2opt = COCOB(s2.parameters()) if list(s2.parameters()) else None
        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        if self.dae_steps:

            torch.manual_seed(1)
            X = env.train()[0]
            torch_dataset = torch.utils.data.TensorDataset(X)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.L1Loss(reduction='none')
            for _ in range(self.dae_steps):
                for (_X,) in torch_loader:
                    if s0opt: s0opt.zero_grad()
                    if saopt: saopt.zero_grad()
                    loss(sa(s0(_X.nan_to_num())),_X).nanmean().backward()
                    if s0opt: s0opt.step()
                    if saopt: saopt.step()

        if self.ssl_samps:

            if self.ssl_neg == 0: raise Exception("neg can't be 0")
            if self.ssl_tau == 0: raise Exception("Tau can't be 0")

            torch.manual_seed(1)

            for A, P, N in env.ssl(self.ssl_neg,self.ssl_samps):

                torch_dataset = torch.utils.data.TensorDataset(A,P,N)
                torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=64,drop_last=True,shuffle=True)

                for _A, _P, _N in torch_loader:
                    #https://arxiv.org/pdf/2002.05709
                    _A = sb(s1(s0(_A.nan_to_num())))
                    _P = sb(s1(s0(_P.nan_to_num())))
                    _N = sb(s1(s0(_N.nan_to_num())))

                    p = torch.einsum("bi,bi->b",_A,_P)
                    n = torch.einsum("bi,bji->bj",_A,_N)

                    p /= (torch.linalg.norm(_A,dim=1)*torch.linalg.norm(_P,dim=1))
                    n /= (torch.linalg.norm(_A,dim=1).unsqueeze(1)*torch.linalg.norm(_N,dim=2))

                    p = torch.exp(p/self.ssl_tau)
                    n = torch.exp(n/self.ssl_tau)

                    if s1opt: s1opt.zero_grad()
                    if sbopt: sbopt.zero_grad()
                    (-torch.log(p/(p+n.sum(dim=1)))).mean().backward()
                    if s1opt: s1opt.step()
                    if sbopt: sbopt.step()

        if self.ws_steps:

            if s0opt: s0opt.zero_grad()
            if s1opt: s1opt.zero_grad()
            if s2opt: s2opt.zero_grad()
            if s3opt: s3opt.zero_grad()

            torch.manual_seed(1)
            
            X, Y = env.train()
            Y = Y[:,self.y]
            
            torch_dataset = torch.utils.data.TensorDataset(X,Y)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.BCEWithLogitsLoss()
            for _ in range(self.ws_steps):
                for _X,_y in torch_loader:

                    if s0opt: s0opt.zero_grad()
                    if s1opt: s1opt.zero_grad()
                    if s2opt: s2opt.zero_grad()
                    if s3opt: s3opt.zero_grad()
                    loss(s3(s2(s1(s0(_X.nan_to_num())))),_y).backward()
                    if s3opt: s3opt.step()
                    if s2opt: s2opt.step()
                    if s1opt: s1opt.step()
                    if s0opt: s0opt.step()

        s0.eval()
        s1.eval()
        s2.eval()
        s3.eval()

        torch.manual_seed(1)

        X, Y = env.test()
        Y = Y[:,self.y]

        rng = cb.CobaRandom(1)
        while 1 in [len(set(Y[35:,i].tolist())) for i in range(Y.shape[1])]:
            ind = rng.shuffle(range(len(Y)))
            X = X[ind,:]
            Y = Y[ind,:]

        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        lrnx = []
        lrny = []
        mems = []

        def predict(X):
            return torch.sigmoid(s3(s2(s1(s0(X.nan_to_num())))))

        def score(X,Y):
            with torch.no_grad():
                return {f'auc{y}': roc_auc_score(Y[:,i],predict(X[:,:])[:,i]) for i,y in enumerate(self.y) }

        yield score(X[35:,:], Y[35:,:])

        loss = torch.nn.BCEWithLogitsLoss()
        for i in range(35):

            x,y = X[i,:], Y[i,:]

            if self.pers_lrn_cnt:
                lrnx.append(x)
                lrny.append(y)

            if self.pers_mem_cnt: 
                mems.append([x,y,self.pers_mem_rpt])

            if len(mems) > self.pers_mem_cnt:
                rng.shuffle(mems, inplace=True)
                for j in reversed(range(self.pers_mem_rcl)):
                    x,y,n = mems[j]
                    lrnx.append(x)
                    lrny.append(y)
                    if n == 1: mems.pop(j)
                    else: mems[j] = [x,y,n-1]

            if len(lrnx) >= self.pers_lrn_cnt:
                x = torch.stack(lrnx[:self.pers_lrn_cnt])
                y = torch.stack(lrny[:self.pers_lrn_cnt])
                if s3opt: s3opt.zero_grad()
                loss(s3(s2(s1(s0(x.nan_to_num())))),y).backward()
                if s3opt: s3opt.step()
                lrnx = lrnx[self.pers_lrn_cnt:]
                lrny = lrny[self.pers_lrn_cnt:]

            yield score(X[35:,:], Y[35:,:])

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if 1 in [len(set(tst_y[:,i].tolist())) for i in range(tst_y.shape[1])]:
                all_equal.add(g)
                continue

            envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))

    if len(all_equal) > 0:
        print(f"All Equal, no environment added for {sorted(all_equal)}")

    return envs

r = lambda w:  ['l', 'r', w, 'l', 'r', w]

lrns = [ FeedForward([]) ]

vals = lambda x: [
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90 ,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [0]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90 ,30,'l','r'), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90 ,30,'l','r'), (30,2), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [0,1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90 ,30,'l','r'), (30,2), 2, 3, 0, 0, 0, 0, 4, 3, 20, 2, 2, [0,1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90 ,30,'l','r'), (30,2), 2, 3, 0, 0, 0, 0, 5, 3, 20, 2, 2, [0,1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90 ,45,'l','r'), (45,2), 2, 3, 0, 0, 0, 0, 4, 3, 20, 2, 2, [0,1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90 ,45,'l','r'), (45,2), 2, 3, 0, 0, 0, 0, 5, 3, 20, 2, 2, [0,1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90 ,30,'l','r'), (30,2), 2, 3, 0, 0, 0, 0, 6, 3, 20, 2, 2, [0,1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90 ,30,'l','r'), (30,2), 2, 3, 0, 0, 0, 0, 7, 3, 20, 2, 2, [0,1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90 ,45,'l','r'), (45,2), 2, 3, 0, 0, 0, 0, 6, 3, 20, 2, 2, [0,1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90 ,45,'l','r'), (45,2), 2, 3, 0, 0, 0, 0, 7, 3, 20, 2, 2, [0,1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90 ,90,'l','r'), (90,2), 2, 3, 0, 0, 0, 0, 7, 3, 20, 2, 2, [0,1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90 ,45,'l','r'), (45,1), 2, 3, 0, 0, 0, 0, 7, 3, 20, 2, 2, [1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90 ,45,'l','r'), (45,2), 2, 3, 0, 0, 0, 0, 10, 3, 20, 2, 2, [0,1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90 ,90,'l','r'), (90,2), 2, 3, 0, 0, 0, 0, 10, 3, 20, 2, 2, [0,1]),
]

envs = make_envs(X3,np.hstack([Y1,Y2]),G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/5/out5_5.log',processes=35,quiet=True)

All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 447, 451, 455, 469]
{'Learners': 1, 'Environments': 226, 'Interactions': 122040}


In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr,i):
        from itertools import compress, repeat, chain
        from operator import eq

        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.train_X)))

        X = self.train_X.tolist()    
        Y = self.train_Y[:,i]
        Y = list(map(tuple,Y.tolist()))

        X = list(map(X.__getitem__,rng_indexes))
        Y = list(map(Y.__getitem__,rng_indexes))

        eq_class  = {y: list(compress(X,map(eq,Y,repeat(y)))) for y in set(Y)}
        ne_class  = {y: list(chain(*[v for k,v in eq_class.items() if k != y ])) for y in set(Y)}

        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        if sr < 1:
            anchor, positive, negative = [], [], []
            for i in range(int(len(X)*sr)):
                x,y = X[i],Y[i]
                anchor.append(x)
                positive.append(choose_unique(eq_class[y],x))
                negative.append(choose_n     (ne_class[y],neg))
            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

        else:
            for _ in range(sr):
                anchor, positive, negative = [], [], []
                for x,y in zip(X,Y):
                    anchor.append(x)
                    positive.append(choose_unique(eq_class[y],x))
                    negative.append(choose_n     (ne_class[y],neg))

                yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s0, s1, s2, s3, dae_steps, dae_dropn, ssl_samps, ssl_neg, ssl_dropn, ssl_tau, ws_steps, pers_lrn_cnt, pers_mem_cnt, pers_mem_rpt, pers_mem_rcl, y):

        self.s0  = s0  #dae + sl
        self.s1  = s1  #ssl + sl
        self.s2  = s2  #sl
        self.s3  = s3  #sl + pers

        self.dae_steps = dae_steps
        self.dae_dropn = dae_dropn

        self.ssl_samps = ssl_samps
        self.ssl_neg   = ssl_neg
        self.ssl_tau   = ssl_tau
        self.ssl_dropn = ssl_dropn

        self.ws_steps = ws_steps

        self.pers_lrn_cnt = pers_lrn_cnt
        self.pers_mem_cnt = pers_mem_cnt
        self.pers_mem_rpt = pers_mem_rpt
        self.pers_mem_rcl = pers_mem_rcl

        self.y = y

        self.params = {'s0': s0, 's1': s1, 's2':s2, 's3':s3, 'dae': (dae_steps,dae_dropn), 'ssl': (ssl_samps,ssl_neg,ssl_dropn,ssl_tau), 'ws': ws_steps, 'pers': (pers_lrn_cnt,pers_mem_cnt,pers_mem_rpt,pers_mem_rcl), 'y': y}

    def evaluate(self, env, lrn):
        from sklearn.metrics import roc_auc_score

        torch.set_num_threads(1)
        torch.manual_seed(1)

        s0 = FeedForward(self.s0)
        s1 = FeedForward(self.s1)
        s2 = FeedForward(self.s2)
        s3 = FeedForward(self.s3)

        sa = torch.nn.Sequential(*list(s0.children())[len(self.s0)-self.dae_dropn:])
        s0 = torch.nn.Sequential(*list(s0.children())[:len(self.s0)-self.dae_dropn])

        sb = torch.nn.Sequential(*list(s1.children())[len(self.s1)-self.ssl_dropn:])
        s1 = torch.nn.Sequential(*list(s1.children())[:len(self.s1)-self.ssl_dropn])

        s0.train()
        sa.train()
        s1.train()
        sb.train()
        s2.train()
        s3.train()

        s0opt = COCOB(s0.parameters()) if list(s0.parameters()) else None
        saopt = COCOB(sa.parameters()) if list(sa.parameters()) else None
        s1opt = COCOB(s1.parameters()) if list(s1.parameters()) else None
        sbopt = COCOB(sb.parameters()) if list(sb.parameters()) else None
        s2opt = COCOB(s2.parameters()) if list(s2.parameters()) else None
        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        if self.dae_steps:

            torch.manual_seed(1)
            X = env.train()[0]
            torch_dataset = torch.utils.data.TensorDataset(X)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.L1Loss(reduction='none')
            for _ in range(self.dae_steps):
                for (_X,) in torch_loader:
                    if s0opt: s0opt.zero_grad()
                    if saopt: saopt.zero_grad()
                    loss(sa(s0(_X.nan_to_num())),_X).nanmean().backward()
                    if s0opt: s0opt.step()
                    if saopt: saopt.step()

        if self.ssl_samps:

            if self.ssl_neg == 0: raise Exception("neg can't be 0")
            if self.ssl_tau == 0: raise Exception("Tau can't be 0")

            torch.manual_seed(1)

            for A, P, N in env.ssl(self.ssl_neg,self.ssl_samps,self.y):

                torch_dataset = torch.utils.data.TensorDataset(A,P,N)
                torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=64,drop_last=True,shuffle=True)

                for _A, _P, _N in torch_loader:
                    #https://arxiv.org/pdf/2002.05709
                    _A = sb(s1(s0(_A.nan_to_num())))
                    _P = sb(s1(s0(_P.nan_to_num())))
                    _N = sb(s1(s0(_N.nan_to_num())))

                    p = torch.einsum("bi,bi->b",_A,_P)
                    n = torch.einsum("bi,bji->bj",_A,_N)

                    p /= (torch.linalg.norm(_A,dim=1)*torch.linalg.norm(_P,dim=1))
                    n /= (torch.linalg.norm(_A,dim=1).unsqueeze(1)*torch.linalg.norm(_N,dim=2))

                    p = torch.exp(p/self.ssl_tau)
                    n = torch.exp(n/self.ssl_tau)

                    if s1opt: s1opt.zero_grad()
                    if sbopt: sbopt.zero_grad()
                    (-torch.log(p/(p+n.sum(dim=1)))).mean().backward()
                    if s1opt: s1opt.step()
                    if sbopt: sbopt.step()

        if self.ws_steps:

            if s0opt: s0opt.zero_grad()
            if s1opt: s1opt.zero_grad()
            if s2opt: s2opt.zero_grad()
            if s3opt: s3opt.zero_grad()

            torch.manual_seed(1)
            
            X, Y = env.train()
            Y = Y[:,self.y]
            
            torch_dataset = torch.utils.data.TensorDataset(X,Y)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.BCEWithLogitsLoss()
            for _ in range(self.ws_steps):
                for _X,_y in torch_loader:

                    if s0opt: s0opt.zero_grad()
                    if s1opt: s1opt.zero_grad()
                    if s2opt: s2opt.zero_grad()
                    if s3opt: s3opt.zero_grad()
                    loss(s3(s2(s1(s0(_X.nan_to_num())))),_y).backward()
                    if s3opt: s3opt.step()
                    if s2opt: s2opt.step()
                    if s1opt: s1opt.step()
                    if s0opt: s0opt.step()

        s0.eval()
        s1.eval()
        s2.eval()
        s3.eval()

        torch.manual_seed(1)

        X, Y = env.test()
        Y = Y[:,self.y]

        rng = cb.CobaRandom(1)
        while 1 in [len(set(Y[35:,i].tolist())) for i in range(Y.shape[1])]:
            ind = rng.shuffle(range(len(Y)))
            X = X[ind,:]
            Y = Y[ind,:]

        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        lrnx = []
        lrny = []
        mems = []

        def predict(X):
            return torch.sigmoid(s3(s2(s1(s0(X.nan_to_num())))))

        def score(X,Y):
            with torch.no_grad():
                return {f'auc{y}': roc_auc_score(Y[:,i],predict(X[:,:])[:,i]) for i,y in enumerate(self.y) }

        yield score(X[35:,:], Y[35:,:])

        loss = torch.nn.BCEWithLogitsLoss()
        for i in range(35):

            x,y = X[i,:], Y[i,:]

            if self.pers_lrn_cnt:
                lrnx.append(x)
                lrny.append(y)

            if self.pers_mem_cnt: 
                mems.append([x,y,self.pers_mem_rpt])

            if len(mems) > self.pers_mem_cnt:
                rng.shuffle(mems, inplace=True)
                for j in reversed(range(self.pers_mem_rcl)):
                    x,y,n = mems[j]
                    lrnx.append(x)
                    lrny.append(y)
                    if n == 1: mems.pop(j)
                    else: mems[j] = [x,y,n-1]

            if len(lrnx) >= self.pers_lrn_cnt:
                x = torch.stack(lrnx[:self.pers_lrn_cnt])
                y = torch.stack(lrny[:self.pers_lrn_cnt])
                if s3opt: s3opt.zero_grad()
                loss(s3(s2(s1(s0(x.nan_to_num())))),y).backward()
                if s3opt: s3opt.step()
                lrnx = lrnx[self.pers_lrn_cnt:]
                lrny = lrny[self.pers_lrn_cnt:]

            yield score(X[35:,:], Y[35:,:])

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if 1 in [len(set(tst_y[:,i].tolist())) for i in range(tst_y.shape[1])]:
                all_equal.add(g)
                continue

            envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))

    if len(all_equal) > 0:
        print(f"All Equal, no environment added for {sorted(all_equal)}")

    return envs

r = lambda w:  ['l', 'r', w, 'l', 'r', w]

lrns = [ FeedForward([]) ]

vals = lambda x: [
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (90,30,'l','r',30,'l','r',30,'l','r'), (), (30,1), 2, 3, 1, 5, 3, 1, 3, 3, 20, 2, 2, [1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (90,30,'l','r',30,'l','r',30,'l','r'), (), (30,1), 2, 3, 2, 5, 3, 1, 3, 3, 20, 2, 2, [1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (90,30,'l','r',30,'l','r',30,'l','r'), (), (30,1), 2, 3, 0, 5, 3, 1, 3, 3, 20, 2, 2, [1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (90,30,'l','r',30,'l','r',30,'l','r'), (), (30,1), 2, 3, .5, 5, 3, 1, 3, 3, 20, 2, 2, [1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (90,30,'l','r',30,'l','r',30,'l','r'), (), (30,1), 2, 3, .5, 5, 6, 1, 3, 3, 20, 2, 2, [1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (90,30,'l','r',), (), (30,1), 2, 3, .5, 5, 0, 1, 3, 3, 20, 2, 2, [1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (90,30,'l','r',), (), (30,1), 2, 3, .25, 5, 0, 1, 3, 3, 20, 2, 2, [1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (90,30,'l','r',), (), (30,1), 2, 3, .75, 5, 0, 1, 3, 3, 20, 2, 2, [1]),    
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (90,30,'l','r',), (), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (90,30,'l','r',30,'l','r',), (), (30,1), 2, 3, .25, 5, 3, 1, 3, 3, 20, 2, 2, [1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (90,30,'l','r',30,'l','r',), (), (30,1), 2, 3, .50, 5, 3, 1, 3, 3, 20, 2, 2, [1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (90,30,'l','r',30,'l','r',), (), (30,1), 2, 3, .75, 5, 3, 1, 3, 3, 20, 2, 2, [1]),

    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (90,45,'l','r',), (), (45,1), 2, 3, .5, 5, 0, 1, 3, 3, 20, 2, 2, [1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (90,90,'l','r',), (), (90,1), 2, 3, .5, 5, 0, 1, 3, 3, 20, 2, 2, [1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (90,90,'l','r',), (), (90,1), 2, 3, .5, 5, 0, 1, 3, 5, 20, 2, 2, [1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (90,45,'l','r',), (), (45,1), 2, 3, .25, 5, 0, 1, 3, 3, 20, 2, 2, [1]),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (90,45,'l','r',), (), (45,1), 2, 3, .75, 5, 0, 1, 3, 3, 20, 2, 2, [1]),
]

envs = make_envs(X3,np.hstack([Y1,Y2]),G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/5/out5_6.log',processes=35,quiet=True)

All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 447, 451, 455, 469]
{'Learners': 1, 'Environments': 226, 'Interactions': 146448}


In [ ]:
import coba as cb

class EMT:

    def __init__(self, split:int = 100, scorer:str="self_consistent_rank", bound:int=0, features=[1,'x']) -> None:

        self.params = {'split':split, 'scorer':scorer, 'bound':bound, 'features':features}

        feat_args = []
        if 1   not in features: feat_args.append('--noconstant')
        if 'a' not in features: feat_args.append('--ignore_linear a')
        if 'x' not in features: feat_args.append('--ignore_linear x')
        feat_args += [ f'--interactions {f}' for f in features if f not in {1, 'a', 'x'} ]

        vw_args = [
            "--emt",
            f"--emt_tree {bound}",
            f"--emt_leaf {split}",
            f"--emt_scorer {scorer}",
            f"--emt_router {'eigen'}",
            f"-b {26}",
            "--min_prediction 0",
            "--max_prediction 3",
            "--coin",
            "--initial_weight 0",
            *feat_args,
            '--quiet',
            '--random_seed 1337'
        ]

        self._vw = cb.VowpalMediator()
        self._vw_args = ' '.join(vw_args)

    def predict(self, X):
        if not self._vw.is_initialized: self._vw.init_learner(self._vw_args, label_type=2)
        return torch.tensor([[float(self._vw.predict(self._vw.make_example({'x':x}, None)))] for x in X.tolist()])

    def learn(self, X, Y):
        if not self._vw.is_initialized: self._vw.init_learner(self._vw_args, label_type=2)
        for x,y in zip(X.tolist(),Y.squeeze(1).tolist()):
            self._vw.learn(self._vw.make_example({'x':x}, f"{int(y)} 1"))

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr,i):
        from itertools import compress, repeat, chain
        from operator import eq

        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.train_X)))

        X = self.train_X.tolist()    
        Y = self.train_Y[:,i]
        Y = list(map(tuple,Y.tolist()))

        X = list(map(X.__getitem__,rng_indexes))
        Y = list(map(Y.__getitem__,rng_indexes))

        eq_class  = {y: list(compress(X,map(eq,Y,repeat(y)))) for y in set(Y)}
        ne_class  = {y: list(chain(*[v for k,v in eq_class.items() if k != y ])) for y in set(Y)}

        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        if sr < 1:
            anchor, positive, negative = [], [], []
            for i in range(int(len(X)*sr)):
                x,y = X[i],Y[i]
                anchor.append(x)
                positive.append(choose_unique(eq_class[y],x))
                negative.append(choose_n     (ne_class[y],neg))
            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

        else:
            for _ in range(sr):
                anchor, positive, negative = [], [], []
                for x,y in zip(X,Y):
                    anchor.append(x)
                    positive.append(choose_unique(eq_class[y],x))
                    negative.append(choose_n     (ne_class[y],neg))

                yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s0, s1, s2, s3, dae_steps, dae_dropn, ssl_samps, ssl_neg, ssl_dropn, ssl_tau, ws_steps, pers_lrn_cnt, pers_mem_cnt, pers_mem_rpt, pers_mem_rcl, y, emt_cnt, emt_bound):

        self.s0  = s0  #dae + sl
        self.s1  = s1  #ssl + sl
        self.s2  = s2  #sl
        self.s3  = s3  #sl + pers

        self.dae_steps = dae_steps
        self.dae_dropn = dae_dropn

        self.ssl_samps = ssl_samps
        self.ssl_neg   = ssl_neg
        self.ssl_tau   = ssl_tau
        self.ssl_dropn = ssl_dropn

        self.ws_steps = ws_steps

        self.pers_lrn_cnt = pers_lrn_cnt
        self.pers_mem_cnt = pers_mem_cnt
        self.pers_mem_rpt = pers_mem_rpt
        self.pers_mem_rcl = pers_mem_rcl

        self.emt_cnt = emt_cnt
        self.emt_bound = emt_bound

        self.y = y

        self.params = {'s0': s0, 's1': s1, 's2':s2, 's3':s3, 'dae': (dae_steps,dae_dropn), 'ssl': (ssl_samps,ssl_neg,ssl_dropn,ssl_tau), 'ws': ws_steps, 'pers': (pers_lrn_cnt,pers_mem_cnt,pers_mem_rpt,pers_mem_rcl), 'y': y, 'emt':(emt_cnt,emt_bound)}

    def evaluate(self, env, lrn):
        from sklearn.metrics import roc_auc_score

        torch.set_num_threads(1)
        torch.manual_seed(1)

        emt = EMT(bound=self.emt_bound)

        s0 = FeedForward(self.s0)
        s1 = FeedForward(self.s1)
        s2 = FeedForward(self.s2)
        s3 = FeedForward(self.s3)

        sa = torch.nn.Sequential(*list(s0.children())[len(self.s0)-self.dae_dropn:])
        s0 = torch.nn.Sequential(*list(s0.children())[:len(self.s0)-self.dae_dropn])

        sb = torch.nn.Sequential(*list(s1.children())[len(self.s1)-self.ssl_dropn:])
        s1 = torch.nn.Sequential(*list(s1.children())[:len(self.s1)-self.ssl_dropn])

        s0.train()
        sa.train()
        s1.train()
        sb.train()
        s2.train()
        s3.train()

        s0opt = COCOB(s0.parameters()) if list(s0.parameters()) else None
        saopt = COCOB(sa.parameters()) if list(sa.parameters()) else None
        s1opt = COCOB(s1.parameters()) if list(s1.parameters()) else None
        sbopt = COCOB(sb.parameters()) if list(sb.parameters()) else None
        s2opt = COCOB(s2.parameters()) if list(s2.parameters()) else None
        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        if self.dae_steps:

            torch.manual_seed(1)
            X = env.train()[0]
            torch_dataset = torch.utils.data.TensorDataset(X)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.L1Loss(reduction='none')
            for _ in range(self.dae_steps):
                for (_X,) in torch_loader:
                    if s0opt: s0opt.zero_grad()
                    if saopt: saopt.zero_grad()
                    loss(sa(s0(_X.nan_to_num())),_X).nanmean().backward()
                    if s0opt: s0opt.step()
                    if saopt: saopt.step()

        if self.ssl_samps:

            if self.ssl_neg == 0: raise Exception("neg can't be 0")
            if self.ssl_tau == 0: raise Exception("Tau can't be 0")

            torch.manual_seed(1)

            for A, P, N in env.ssl(self.ssl_neg,self.ssl_samps,self.y):

                torch_dataset = torch.utils.data.TensorDataset(A,P,N)
                torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=64,drop_last=True,shuffle=True)

                for _A, _P, _N in torch_loader:
                    #https://arxiv.org/pdf/2002.05709
                    _A = sb(s1(s0(_A.nan_to_num())))
                    _P = sb(s1(s0(_P.nan_to_num())))
                    _N = sb(s1(s0(_N.nan_to_num())))

                    p = torch.einsum("bi,bi->b",_A,_P)
                    n = torch.einsum("bi,bji->bj",_A,_N)

                    p /= (torch.linalg.norm(_A,dim=1)*torch.linalg.norm(_P,dim=1))
                    n /= (torch.linalg.norm(_A,dim=1).unsqueeze(1)*torch.linalg.norm(_N,dim=2))

                    p = torch.exp(p/self.ssl_tau)
                    n = torch.exp(n/self.ssl_tau)

                    if s1opt: s1opt.zero_grad()
                    if sbopt: sbopt.zero_grad()
                    (-torch.log(p/(p+n.sum(dim=1)))).mean().backward()
                    if s1opt: s1opt.step()
                    if sbopt: sbopt.step()

        if self.ws_steps:

            if s0opt: s0opt.zero_grad()
            if s1opt: s1opt.zero_grad()
            if s2opt: s2opt.zero_grad()
            if s3opt: s3opt.zero_grad()

            torch.manual_seed(1)
            
            X, Y = env.train()
            Y = Y[:,self.y]
            
            torch_dataset = torch.utils.data.TensorDataset(X,Y)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.BCEWithLogitsLoss()
            for _ in range(self.ws_steps):
                for _X,_y in torch_loader:

                    if s0opt: s0opt.zero_grad()
                    if s1opt: s1opt.zero_grad()
                    if s2opt: s2opt.zero_grad()
                    if s3opt: s3opt.zero_grad()
                    loss(s3(s2(s1(s0(_X.nan_to_num())))),_y).backward()
                    if s3opt: s3opt.step()
                    if s2opt: s2opt.step()
                    if s1opt: s1opt.step()
                    if s0opt: s0opt.step()

            rng = cb.CobaRandom(1)
            ind = rng.shuffle(range(len(Y)))[:self.emt_cnt]
            X = X[ind,:]
            Y = Y[ind,:]

            emt.learn(s2(s1(s0(X.nan_to_num()))),Y)

        s0.eval()
        s1.eval()
        s2.eval()
        s3.eval()

        torch.manual_seed(1)

        X, Y = env.test()
        Y = Y[:,self.y]

        rng = cb.CobaRandom(1)
        while 1 in [len(set(Y[35:,i].tolist())) for i in range(Y.shape[1])]:
            ind = rng.shuffle(range(len(Y)))
            X = X[ind,:]
            Y = Y[ind,:]

        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        lrnx = []
        lrny = []
        mems = []

        def predict(X):
            return emt.predict(s2(s1(s0(X.nan_to_num()))))

        def score(X,Y):
            with torch.no_grad():
                return {f'auc{y}': roc_auc_score(Y[:,i],predict(X[:,:])[:,i]) for i,y in enumerate(self.y) }

        yield score(X[35:,:], Y[35:,:])

        loss = torch.nn.BCEWithLogitsLoss()
        for i in range(35):

            x,y = X[[i],:], Y[[i],:]

            emt.learn(s2(s1(s0(x.nan_to_num()))),y)

            yield score(X[35:,:], Y[35:,:])

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if 1 in [len(set(tst_y[:,i].tolist())) for i in range(tst_y.shape[1])]:
                all_equal.add(g)
                continue

            envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))

    if len(all_equal) > 0:
        print(f"All Equal, no environment added for {sorted(all_equal)}")

    return envs

lrns = [ FeedForward([]) ]
vals = lambda x: [
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 5000, 100),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 10000, 100),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 100),
]
envs = make_envs(X3,np.hstack([Y1,Y2]),G)

cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/5/out5_7.log',processes=35,quiet=True)

All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 447, 451, 455, 469]
{'Learners': 1, 'Environments': 92, 'Interactions': 8676}


In [ ]:
import coba as cb

class EMT:

    def __init__(self, split:int = 100, scorer:str="self_consistent_rank", bound:int=0, features=[1,'x']) -> None:

        self.params = {'split':split, 'scorer':scorer, 'bound':bound, 'features':features}

        feat_args = []
        if 1   not in features: feat_args.append('--noconstant')
        if 'a' not in features: feat_args.append('--ignore_linear a')
        if 'x' not in features: feat_args.append('--ignore_linear x')
        feat_args += [ f'--interactions {f}' for f in features if f not in {1, 'a', 'x'} ]

        vw_args = [
            "--emt",
            f"--emt_tree {bound}",
            f"--emt_leaf {split}",
            f"--emt_scorer {scorer}",
            f"--emt_router {'eigen'}",
            f"-b {26}",
            "--min_prediction 0",
            "--max_prediction 3",
            "--coin",
            "--initial_weight 0",
            *feat_args,
            '--quiet',
            '--random_seed 1337'
        ]

        self._vw = cb.VowpalMediator()
        self._vw_args = ' '.join(vw_args)

    def predict(self, X):
        if not self._vw.is_initialized: self._vw.init_learner(self._vw_args, label_type=2)
        return torch.tensor([[float(self._vw.predict(self._vw.make_example({"x":x}, None)))] for x in X.tolist()])

    def learn(self, X, Y):
        if not self._vw.is_initialized: self._vw.init_learner(self._vw_args, label_type=2)
        for x,y in zip(X.tolist(),Y.squeeze(1).tolist()):
            self._vw.learn(self._vw.make_example({'x':x}, f"{int(y)} 1"))

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr,i):
        from itertools import compress, repeat, chain
        from operator import eq

        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.train_X)))

        X = self.train_X.tolist()    
        Y = self.train_Y[:,i]
        Y = list(map(tuple,Y.tolist()))

        X = list(map(X.__getitem__,rng_indexes))
        Y = list(map(Y.__getitem__,rng_indexes))

        eq_class  = {y: list(compress(X,map(eq,Y,repeat(y)))) for y in set(Y)}
        ne_class  = {y: list(chain(*[v for k,v in eq_class.items() if k != y ])) for y in set(Y)}

        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        if sr < 1:
            anchor, positive, negative = [], [], []
            for i in range(int(len(X)*sr)):
                x,y = X[i],Y[i]
                anchor.append(x)
                positive.append(choose_unique(eq_class[y],x))
                negative.append(choose_n     (ne_class[y],neg))
            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

        else:
            for _ in range(sr):
                anchor, positive, negative = [], [], []
                for x,y in zip(X,Y):
                    anchor.append(x)
                    positive.append(choose_unique(eq_class[y],x))
                    negative.append(choose_n     (ne_class[y],neg))

                yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s0, s1, s2, s3, dae_steps, dae_dropn, ssl_samps, ssl_neg, ssl_dropn, ssl_tau, ws_steps, pers_lrn_cnt, pers_mem_cnt, pers_mem_rpt, pers_mem_rcl, y, emt_cnt, emt_bound, emt_feats):

        self.s0  = s0  #dae + sl
        self.s1  = s1  #ssl + sl
        self.s2  = s2  #sl
        self.s3  = s3  #sl + pers

        self.dae_steps = dae_steps
        self.dae_dropn = dae_dropn

        self.ssl_samps = ssl_samps
        self.ssl_neg   = ssl_neg
        self.ssl_tau   = ssl_tau
        self.ssl_dropn = ssl_dropn

        self.ws_steps = ws_steps

        self.pers_lrn_cnt = pers_lrn_cnt
        self.pers_mem_cnt = pers_mem_cnt
        self.pers_mem_rpt = pers_mem_rpt
        self.pers_mem_rcl = pers_mem_rcl

        self.emt_cnt = emt_cnt
        self.emt_bound = emt_bound
        self.emt_feats = emt_feats

        self.y = y

        self.params = {'s0': s0, 's1': s1, 's2':s2, 's3':s3, 'dae': (dae_steps,dae_dropn), 'ssl': (ssl_samps,ssl_neg,ssl_dropn,ssl_tau), 'ws': ws_steps, 'pers': (pers_lrn_cnt,pers_mem_cnt,pers_mem_rpt,pers_mem_rcl), 'y': y, 'emt':(emt_cnt,emt_bound,emt_feats)}

    def evaluate(self, env, lrn):
        from sklearn.metrics import roc_auc_score

        torch.set_num_threads(1)
        torch.manual_seed(1)

        emt = EMT(bound=self.emt_bound,features=self.emt_feats)

        s0 = FeedForward(self.s0)
        s1 = FeedForward(self.s1)
        s2 = FeedForward(self.s2)
        s3 = FeedForward(self.s3)

        sa = torch.nn.Sequential(*list(s0.children())[len(self.s0)-self.dae_dropn:])
        s0 = torch.nn.Sequential(*list(s0.children())[:len(self.s0)-self.dae_dropn])

        sb = torch.nn.Sequential(*list(s1.children())[len(self.s1)-self.ssl_dropn:])
        s1 = torch.nn.Sequential(*list(s1.children())[:len(self.s1)-self.ssl_dropn])

        s0.train()
        sa.train()
        s1.train()
        sb.train()
        s2.train()
        s3.train()

        s0opt = COCOB(s0.parameters()) if list(s0.parameters()) else None
        saopt = COCOB(sa.parameters()) if list(sa.parameters()) else None
        s1opt = COCOB(s1.parameters()) if list(s1.parameters()) else None
        sbopt = COCOB(sb.parameters()) if list(sb.parameters()) else None
        s2opt = COCOB(s2.parameters()) if list(s2.parameters()) else None
        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        if self.dae_steps:

            torch.manual_seed(1)
            X = env.train()[0]
            torch_dataset = torch.utils.data.TensorDataset(X)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.L1Loss(reduction='none')
            for _ in range(self.dae_steps):
                for (_X,) in torch_loader:
                    if s0opt: s0opt.zero_grad()
                    if saopt: saopt.zero_grad()
                    loss(sa(s0(_X.nan_to_num())),_X).nanmean().backward()
                    if s0opt: s0opt.step()
                    if saopt: saopt.step()

        if self.ssl_samps:

            if self.ssl_neg == 0: raise Exception("neg can't be 0")
            if self.ssl_tau == 0: raise Exception("Tau can't be 0")

            torch.manual_seed(1)

            for A, P, N in env.ssl(self.ssl_neg,self.ssl_samps,self.y):

                torch_dataset = torch.utils.data.TensorDataset(A,P,N)
                torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=64,drop_last=True,shuffle=True)

                for _A, _P, _N in torch_loader:
                    #https://arxiv.org/pdf/2002.05709
                    _A = sb(s1(s0(_A.nan_to_num())))
                    _P = sb(s1(s0(_P.nan_to_num())))
                    _N = sb(s1(s0(_N.nan_to_num())))

                    p = torch.einsum("bi,bi->b",_A,_P)
                    n = torch.einsum("bi,bji->bj",_A,_N)

                    p /= (torch.linalg.norm(_A,dim=1)*torch.linalg.norm(_P,dim=1))
                    n /= (torch.linalg.norm(_A,dim=1).unsqueeze(1)*torch.linalg.norm(_N,dim=2))

                    p = torch.exp(p/self.ssl_tau)
                    n = torch.exp(n/self.ssl_tau)

                    if s1opt: s1opt.zero_grad()
                    if sbopt: sbopt.zero_grad()
                    (-torch.log(p/(p+n.sum(dim=1)))).mean().backward()
                    if s1opt: s1opt.step()
                    if sbopt: sbopt.step()

        if self.ws_steps:

            if s0opt: s0opt.zero_grad()
            if s1opt: s1opt.zero_grad()
            if s2opt: s2opt.zero_grad()
            if s3opt: s3opt.zero_grad()

            torch.manual_seed(1)
            
            X, Y = env.train()
            Y = Y[:,self.y]
            
            torch_dataset = torch.utils.data.TensorDataset(X,Y)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.BCEWithLogitsLoss()
            for _ in range(self.ws_steps):
                for _X,_y in torch_loader:

                    if s0opt: s0opt.zero_grad()
                    if s1opt: s1opt.zero_grad()
                    if s2opt: s2opt.zero_grad()
                    if s3opt: s3opt.zero_grad()
                    loss(s3(s2(s1(s0(_X.nan_to_num())))),_y).backward()
                    if s3opt: s3opt.step()
                    if s2opt: s2opt.step()
                    if s1opt: s1opt.step()
                    if s0opt: s0opt.step()

            rng = cb.CobaRandom(1)
            ind = rng.shuffle(range(len(Y)))[:self.emt_cnt]
            X = X[ind,:]
            Y = Y[ind,:]

            emt.learn(s2(s1(s0(X.nan_to_num()))),Y)

        s0.eval()
        s1.eval()
        s2.eval()
        s3.eval()

        torch.manual_seed(1)

        X, Y = env.test()
        Y = Y[:,self.y]

        rng = cb.CobaRandom(1)
        while 1 in [len(set(Y[35:,i].tolist())) for i in range(Y.shape[1])]:
            ind = rng.shuffle(range(len(Y)))
            X = X[ind,:]
            Y = Y[ind,:]

        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        lrnx = []
        lrny = []
        mems = []

        def predict(X):
            return emt.predict(s2(s1(s0(X.nan_to_num()))))

        def score(X,Y):
            with torch.no_grad():
                return {f'auc{y}': roc_auc_score(Y[:,i],predict(X[:,:])[:,i]) for i,y in enumerate(self.y) }

        yield score(X[35:,:], Y[35:,:])

        loss = torch.nn.BCEWithLogitsLoss()
        for i in range(35):

            x,y = X[[i],:], Y[[i],:]

            emt.learn(s2(s1(s0(x.nan_to_num()))),y)

            yield score(X[35:,:], Y[35:,:])

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if 1 in [len(set(tst_y[:,i].tolist())) for i in range(tst_y.shape[1])]:
                all_equal.add(g)
                continue

            envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))

    if len(all_equal) > 0:
        print(f"All Equal, no environment added for {sorted(all_equal)}")

    return envs

lrns = [ FeedForward([]) ]
vals = lambda x: [
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 100,['x']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 200,['x']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 300,['x']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 100,['x','xx']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 200,['x','xx']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 300,['x','xx']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',30), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 100,['x']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',30), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 200,['x']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',30), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 300,['x']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',30), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 100,['x','xx']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',30), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 200,['x','xx']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',30), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 300,['x','xx']),
]
envs = make_envs(X3,np.hstack([Y1,Y2]),G)

cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/5/out5_8.log',processes=35,quiet=True)

All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 447, 451, 455, 469]
{'Learners': 1, 'Environments': 223, 'Interactions': 94824}


In [ ]:
import coba as cb

class EMT:

    def __init__(self, split:int = 100, scorer:str="self_consistent_rank", bound:int=0, features=[1,'x']) -> None:

        self.params = {'split':split, 'scorer':scorer, 'bound':bound, 'features':features}

        feat_args = []
        if 1   not in features: feat_args.append('--noconstant')
        if 'a' not in features: feat_args.append('--ignore_linear a')
        if 'x' not in features: feat_args.append('--ignore_linear x')
        feat_args += [ f'--interactions {f}' for f in features if f not in {1, 'a', 'x'} ]

        vw_args = [
            "--emt",
            f"--emt_tree {bound}",
            f"--emt_leaf {split}",
            f"--emt_scorer {scorer}",
            f"--emt_router {'eigen'}",
            f"-b {26}",
            "--min_prediction 0",
            "--max_prediction 3",
            "--coin",
            "--initial_weight 0",
            *feat_args,
            '--quiet',
            '--random_seed 1337'
        ]

        self._vw = cb.VowpalMediator()
        self._vw_args = ' '.join(vw_args)

    def predict(self, X):
        if not self._vw.is_initialized: self._vw.init_learner(self._vw_args, label_type=2)
        return torch.tensor([[float(self._vw.predict(self._vw.make_example({"x":x}, None)))] for x in X.tolist()])

    def learn(self, X, Y):
        if not self._vw.is_initialized: self._vw.init_learner(self._vw_args, label_type=2)
        for x,y in zip(X.tolist(),Y.squeeze(1).tolist()):
            self._vw.learn(self._vw.make_example({'x':x}, f"{int(y)} 1"))

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr,i):
        from itertools import compress, repeat, chain
        from operator import eq

        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.train_X)))

        X = self.train_X.tolist()    
        Y = self.train_Y[:,i]
        Y = list(map(tuple,Y.tolist()))

        X = list(map(X.__getitem__,rng_indexes))
        Y = list(map(Y.__getitem__,rng_indexes))

        eq_class  = {y: list(compress(X,map(eq,Y,repeat(y)))) for y in set(Y)}
        ne_class  = {y: list(chain(*[v for k,v in eq_class.items() if k != y ])) for y in set(Y)}

        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        if sr < 1:
            anchor, positive, negative = [], [], []
            for i in range(int(len(X)*sr)):
                x,y = X[i],Y[i]
                anchor.append(x)
                positive.append(choose_unique(eq_class[y],x))
                negative.append(choose_n     (ne_class[y],neg))
            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

        else:
            for _ in range(sr):
                anchor, positive, negative = [], [], []
                for x,y in zip(X,Y):
                    anchor.append(x)
                    positive.append(choose_unique(eq_class[y],x))
                    negative.append(choose_n     (ne_class[y],neg))

                yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s0, s1, s2, s3, dae_steps, dae_dropn, ssl_samps, ssl_neg, ssl_dropn, ssl_tau, ws_steps, pers_lrn_cnt, pers_mem_cnt, pers_mem_rpt, pers_mem_rcl, y, emt_cnt, emt_bound, emt_feats):

        self.s0  = s0  #dae + sl
        self.s1  = s1  #ssl + sl
        self.s2  = s2  #sl
        self.s3  = s3  #sl + pers

        self.dae_steps = dae_steps
        self.dae_dropn = dae_dropn

        self.ssl_samps = ssl_samps
        self.ssl_neg   = ssl_neg
        self.ssl_tau   = ssl_tau
        self.ssl_dropn = ssl_dropn

        self.ws_steps = ws_steps

        self.pers_lrn_cnt = pers_lrn_cnt
        self.pers_mem_cnt = pers_mem_cnt
        self.pers_mem_rpt = pers_mem_rpt
        self.pers_mem_rcl = pers_mem_rcl

        self.emt_cnt = emt_cnt
        self.emt_bound = emt_bound
        self.emt_feats = emt_feats

        self.y = y

        self.params = {'s0': s0, 's1': s1, 's2':s2, 's3':s3, 'dae': (dae_steps,dae_dropn), 'ssl': (ssl_samps,ssl_neg,ssl_dropn,ssl_tau), 'ws': ws_steps, 'pers': (pers_lrn_cnt,pers_mem_cnt,pers_mem_rpt,pers_mem_rcl), 'y': y, 'emt':(emt_cnt,emt_bound,emt_feats)}

    def evaluate(self, env, lrn):
        from sklearn.metrics import roc_auc_score

        torch.set_num_threads(1)
        torch.manual_seed(1)

        emt = EMT(bound=self.emt_bound,features=self.emt_feats)

        s0 = FeedForward(self.s0)
        s1 = FeedForward(self.s1)
        s2 = FeedForward(self.s2)
        s3 = FeedForward(self.s3)

        sa = torch.nn.Sequential(*list(s0.children())[len(self.s0)-self.dae_dropn:])
        s0 = torch.nn.Sequential(*list(s0.children())[:len(self.s0)-self.dae_dropn])

        sb = torch.nn.Sequential(*list(s1.children())[len(self.s1)-self.ssl_dropn:])
        s1 = torch.nn.Sequential(*list(s1.children())[:len(self.s1)-self.ssl_dropn])

        s0.train()
        sa.train()
        s1.train()
        sb.train()
        s2.train()
        s3.train()

        s0opt = COCOB(s0.parameters()) if list(s0.parameters()) else None
        saopt = COCOB(sa.parameters()) if list(sa.parameters()) else None
        s1opt = COCOB(s1.parameters()) if list(s1.parameters()) else None
        sbopt = COCOB(sb.parameters()) if list(sb.parameters()) else None
        s2opt = COCOB(s2.parameters()) if list(s2.parameters()) else None
        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        if self.dae_steps:

            torch.manual_seed(1)
            X = env.train()[0]
            torch_dataset = torch.utils.data.TensorDataset(X)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.L1Loss(reduction='none')
            for _ in range(self.dae_steps):
                for (_X,) in torch_loader:
                    if s0opt: s0opt.zero_grad()
                    if saopt: saopt.zero_grad()
                    loss(sa(s0(_X.nan_to_num())),_X).nanmean().backward()
                    if s0opt: s0opt.step()
                    if saopt: saopt.step()

        if self.ssl_samps:

            if self.ssl_neg == 0: raise Exception("neg can't be 0")
            if self.ssl_tau == 0: raise Exception("Tau can't be 0")

            torch.manual_seed(1)

            for A, P, N in env.ssl(self.ssl_neg,self.ssl_samps,self.y):

                torch_dataset = torch.utils.data.TensorDataset(A,P,N)
                torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=64,drop_last=True,shuffle=True)

                for _A, _P, _N in torch_loader:
                    #https://arxiv.org/pdf/2002.05709
                    _A = sb(s1(s0(_A.nan_to_num())))
                    _P = sb(s1(s0(_P.nan_to_num())))
                    _N = sb(s1(s0(_N.nan_to_num())))

                    p = torch.einsum("bi,bi->b",_A,_P)
                    n = torch.einsum("bi,bji->bj",_A,_N)

                    p /= (torch.linalg.norm(_A,dim=1)*torch.linalg.norm(_P,dim=1))
                    n /= (torch.linalg.norm(_A,dim=1).unsqueeze(1)*torch.linalg.norm(_N,dim=2))

                    p = torch.exp(p/self.ssl_tau)
                    n = torch.exp(n/self.ssl_tau)

                    if s1opt: s1opt.zero_grad()
                    if sbopt: sbopt.zero_grad()
                    (-torch.log(p/(p+n.sum(dim=1)))).mean().backward()
                    if s1opt: s1opt.step()
                    if sbopt: sbopt.step()

        if self.ws_steps:

            if s0opt: s0opt.zero_grad()
            if s1opt: s1opt.zero_grad()
            if s2opt: s2opt.zero_grad()
            if s3opt: s3opt.zero_grad()

            torch.manual_seed(1)
            
            X, Y = env.train()
            Y = Y[:,self.y]
            
            torch_dataset = torch.utils.data.TensorDataset(X,Y)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.BCEWithLogitsLoss()
            for _ in range(self.ws_steps):
                for _X,_y in torch_loader:

                    if s0opt: s0opt.zero_grad()
                    if s1opt: s1opt.zero_grad()
                    if s2opt: s2opt.zero_grad()
                    if s3opt: s3opt.zero_grad()
                    loss(s3(s2(s1(s0(_X.nan_to_num())))),_y).backward()
                    if s3opt: s3opt.step()
                    if s2opt: s2opt.step()
                    if s1opt: s1opt.step()
                    if s0opt: s0opt.step()

            rng = cb.CobaRandom(1)
            ind = rng.shuffle(range(len(Y)))[:self.emt_cnt]
            X = X[ind,:]
            Y = Y[ind,:]

            emt.learn(s2(s1(s0(X.nan_to_num()))),Y)

        s0.eval()
        s1.eval()
        s2.eval()
        s3.eval()

        torch.manual_seed(1)

        X, Y = env.test()
        Y = Y[:,self.y]

        rng = cb.CobaRandom(1)
        while 1 in [len(set(Y[35:,i].tolist())) for i in range(Y.shape[1])]:
            ind = rng.shuffle(range(len(Y)))
            X = X[ind,:]
            Y = Y[ind,:]

        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        lrnx = []
        lrny = []
        mems = []

        def predict(X):
            return emt.predict(s2(s1(s0(X.nan_to_num()))))

        def score(X,Y):
            with torch.no_grad():
                return {f'auc{y}': roc_auc_score(Y[:,i],predict(X[:,:])[:,i]) for i,y in enumerate(self.y) }

        yield score(X[35:,:], Y[35:,:])

        loss = torch.nn.BCEWithLogitsLoss()
        for i in range(35):

            x,y = X[[i],:], Y[[i],:]

            emt.learn(s2(s1(s0(x.nan_to_num()))),y)

            yield score(X[35:,:], Y[35:,:])

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if 1 in [len(set(tst_y[:,i].tolist())) for i in range(tst_y.shape[1])]:
                all_equal.add(g)
                continue

            envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))

    if len(all_equal) > 0:
        print(f"All Equal, no environment added for {sorted(all_equal)}")

    return envs

lrns = [ FeedForward([]) ]
vals = lambda x: [
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,15,'l','r',), (15,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 50 ,['x']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,15,'l','r',), (15,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 100,['x']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,15,'l','r',), (15,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 150,['x']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 50 ,['x']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 100,['x']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 150,['x']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,45,'l','r',), (45,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 50 ,['x']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,45,'l','r',), (45,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 100,['x']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,45,'l','r',), (45,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 150,['x']),
]
envs = make_envs(X3,np.hstack([Y1,Y2]),G)

cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/5/out5_9.log',processes=35,quiet=True)

All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 447, 451, 455, 469]
{'Learners': 1, 'Environments': 226, 'Interactions': 73224}


In [ ]:
import coba as cb

class EMT:

    def __init__(self, split:int = 100, scorer:str="self_consistent_rank", bound:int=0, features=[1,'x']) -> None:

        self.params = {'split':split, 'scorer':scorer, 'bound':bound, 'features':features}

        feat_args = []
        if 1   not in features: feat_args.append('--noconstant')
        if 'a' not in features: feat_args.append('--ignore_linear a')
        if 'x' not in features: feat_args.append('--ignore_linear x')
        feat_args += [ f'--interactions {f}' for f in features if f not in {1, 'a', 'x'} ]

        vw_args = [
            "--emt",
            f"--emt_tree {bound}",
            f"--emt_leaf {split}",
            f"--emt_scorer {scorer}",
            f"--emt_router {'eigen'}",
            f"-b {26}",
            "--min_prediction 0",
            "--max_prediction 3",
            "--coin",
            "--initial_weight 0",
            *feat_args,
            '--quiet',
            '--random_seed 1337'
        ]

        self._vw = cb.VowpalMediator()
        self._vw_args = ' '.join(vw_args)

    def predict(self, X):
        if not self._vw.is_initialized: self._vw.init_learner(self._vw_args, label_type=2)
        return torch.tensor([[float(self._vw.predict(self._vw.make_example({"x":x}, None)))] for x in X.tolist()])

    def learn(self, X, Y):
        if not self._vw.is_initialized: self._vw.init_learner(self._vw_args, label_type=2)
        for x,y in zip(X.tolist(),Y.squeeze(1).tolist()):
            self._vw.learn(self._vw.make_example({'x':x}, f"{int(y)} 1"))

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr,i):
        from itertools import compress, repeat, chain
        from operator import eq

        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.train_X)))

        X = self.train_X.tolist()    
        Y = self.train_Y[:,i]
        Y = list(map(tuple,Y.tolist()))

        X = list(map(X.__getitem__,rng_indexes))
        Y = list(map(Y.__getitem__,rng_indexes))

        eq_class  = {y: list(compress(X,map(eq,Y,repeat(y)))) for y in set(Y)}
        ne_class  = {y: list(chain(*[v for k,v in eq_class.items() if k != y ])) for y in set(Y)}

        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        if sr < 1:
            anchor, positive, negative = [], [], []
            for i in range(int(len(X)*sr)):
                x,y = X[i],Y[i]
                anchor.append(x)
                positive.append(choose_unique(eq_class[y],x))
                negative.append(choose_n     (ne_class[y],neg))
            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

        else:
            for _ in range(sr):
                anchor, positive, negative = [], [], []
                for x,y in zip(X,Y):
                    anchor.append(x)
                    positive.append(choose_unique(eq_class[y],x))
                    negative.append(choose_n     (ne_class[y],neg))

                yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s0, s1, s2, s3, dae_steps, dae_dropn, ssl_samps, ssl_neg, ssl_dropn, ssl_tau, ws_steps, pers_lrn_cnt, pers_mem_cnt, pers_mem_rpt, pers_mem_rcl, y, emt_cnt, emt_bound, emt_feats):

        self.s0  = s0  #dae + sl
        self.s1  = s1  #ssl + sl
        self.s2  = s2  #sl
        self.s3  = s3  #sl + pers

        self.dae_steps = dae_steps
        self.dae_dropn = dae_dropn

        self.ssl_samps = ssl_samps
        self.ssl_neg   = ssl_neg
        self.ssl_tau   = ssl_tau
        self.ssl_dropn = ssl_dropn

        self.ws_steps = ws_steps

        self.pers_lrn_cnt = pers_lrn_cnt
        self.pers_mem_cnt = pers_mem_cnt
        self.pers_mem_rpt = pers_mem_rpt
        self.pers_mem_rcl = pers_mem_rcl

        self.emt_cnt = emt_cnt
        self.emt_bound = emt_bound
        self.emt_feats = emt_feats

        self.y = y

        self.params = {'s0': s0, 's1': s1, 's2':s2, 's3':s3, 'dae': (dae_steps,dae_dropn), 'ssl': (ssl_samps,ssl_neg,ssl_dropn,ssl_tau), 'ws': ws_steps, 'pers': (pers_lrn_cnt,pers_mem_cnt,pers_mem_rpt,pers_mem_rcl), 'y': y, 'emt':(emt_cnt,emt_bound,emt_feats)}

    def evaluate(self, env, lrn):
        from sklearn.metrics import roc_auc_score

        torch.set_num_threads(1)
        torch.manual_seed(1)

        emt = EMT(bound=self.emt_bound,features=self.emt_feats)

        s0 = FeedForward(self.s0)
        s1 = FeedForward(self.s1)
        s2 = FeedForward(self.s2)
        s3 = FeedForward(self.s3)

        sa = torch.nn.Sequential(*list(s0.children())[len(self.s0)-self.dae_dropn:])
        s0 = torch.nn.Sequential(*list(s0.children())[:len(self.s0)-self.dae_dropn])

        sb = torch.nn.Sequential(*list(s1.children())[len(self.s1)-self.ssl_dropn:])
        s1 = torch.nn.Sequential(*list(s1.children())[:len(self.s1)-self.ssl_dropn])

        s0.train()
        sa.train()
        s1.train()
        sb.train()
        s2.train()
        s3.train()

        s0opt = COCOB(s0.parameters()) if list(s0.parameters()) else None
        saopt = COCOB(sa.parameters()) if list(sa.parameters()) else None
        s1opt = COCOB(s1.parameters()) if list(s1.parameters()) else None
        sbopt = COCOB(sb.parameters()) if list(sb.parameters()) else None
        s2opt = COCOB(s2.parameters()) if list(s2.parameters()) else None
        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        if self.dae_steps:

            torch.manual_seed(1)
            X = env.train()[0]
            torch_dataset = torch.utils.data.TensorDataset(X)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.L1Loss(reduction='none')
            for _ in range(self.dae_steps):
                for (_X,) in torch_loader:
                    if s0opt: s0opt.zero_grad()
                    if saopt: saopt.zero_grad()
                    loss(sa(s0(_X.nan_to_num())),_X).nanmean().backward()
                    if s0opt: s0opt.step()
                    if saopt: saopt.step()

        if self.ssl_samps:

            if self.ssl_neg == 0: raise Exception("neg can't be 0")
            if self.ssl_tau == 0: raise Exception("Tau can't be 0")

            torch.manual_seed(1)

            for A, P, N in env.ssl(self.ssl_neg,self.ssl_samps,self.y):

                torch_dataset = torch.utils.data.TensorDataset(A,P,N)
                torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=64,drop_last=True,shuffle=True)

                for _A, _P, _N in torch_loader:
                    #https://arxiv.org/pdf/2002.05709
                    _A = sb(s1(s0(_A.nan_to_num())))
                    _P = sb(s1(s0(_P.nan_to_num())))
                    _N = sb(s1(s0(_N.nan_to_num())))

                    p = torch.einsum("bi,bi->b",_A,_P)
                    n = torch.einsum("bi,bji->bj",_A,_N)

                    p /= (torch.linalg.norm(_A,dim=1)*torch.linalg.norm(_P,dim=1))
                    n /= (torch.linalg.norm(_A,dim=1).unsqueeze(1)*torch.linalg.norm(_N,dim=2))

                    p = torch.exp(p/self.ssl_tau)
                    n = torch.exp(n/self.ssl_tau)

                    if s1opt: s1opt.zero_grad()
                    if sbopt: sbopt.zero_grad()
                    (-torch.log(p/(p+n.sum(dim=1)))).mean().backward()
                    if s1opt: s1opt.step()
                    if sbopt: sbopt.step()

        if self.ws_steps:

            if s0opt: s0opt.zero_grad()
            if s1opt: s1opt.zero_grad()
            if s2opt: s2opt.zero_grad()
            if s3opt: s3opt.zero_grad()

            torch.manual_seed(1)
            
            X, Y = env.train()
            Y = Y[:,self.y]
            
            torch_dataset = torch.utils.data.TensorDataset(X,Y)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.BCEWithLogitsLoss()
            for _ in range(self.ws_steps):
                for _X,_y in torch_loader:

                    if s0opt: s0opt.zero_grad()
                    if s1opt: s1opt.zero_grad()
                    if s2opt: s2opt.zero_grad()
                    if s3opt: s3opt.zero_grad()
                    loss(s3(s2(s1(s0(_X.nan_to_num())))),_y).backward()
                    if s3opt: s3opt.step()
                    if s2opt: s2opt.step()
                    if s1opt: s1opt.step()
                    if s0opt: s0opt.step()

            rng = cb.CobaRandom(1)
            ind = rng.shuffle(range(len(Y)))[:self.emt_cnt]
            X = X[ind,:]
            Y = Y[ind,:]

            emt.learn(s2(s1(s0(X.nan_to_num()))),Y)

        s0.eval()
        s1.eval()
        s2.eval()
        s3.eval()

        torch.manual_seed(1)

        X, Y = env.test()
        Y = Y[:,self.y]

        rng = cb.CobaRandom(1)
        while 1 in [len(set(Y[35:,i].tolist())) for i in range(Y.shape[1])]:
            ind = rng.shuffle(range(len(Y)))
            X = X[ind,:]
            Y = Y[ind,:]

        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        lrnx = []
        lrny = []
        mems = []

        def predict(X):
            return emt.predict(s2(s1(s0(X.nan_to_num()))))

        def score(X,Y):
            with torch.no_grad():
                return {f'auc{y}': roc_auc_score(Y[:,i],predict(X[:,:])[:,i]) for i,y in enumerate(self.y) }

        yield score(X[35:,:], Y[35:,:])

        loss = torch.nn.BCEWithLogitsLoss()
        for i in range(35):

            x,y = X[[i],:], Y[[i],:]

            emt.learn(s2(s1(s0(x.nan_to_num()))),y)

            yield score(X[35:,:], Y[35:,:])

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if 1 in [len(set(tst_y[:,i].tolist())) for i in range(tst_y.shape[1])]:
                all_equal.add(g)
                continue

            envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))

    if len(all_equal) > 0:
        print(f"All Equal, no environment added for {sorted(all_equal)}")

    return envs

lrns = [ FeedForward([]) ]
vals = lambda x: [
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 50   ,['x']),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 20000, 10000,['x']),
]
envs = make_envs(X3,np.hstack([Y1,Y2]),G)

cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/5/out5_10.log',processes=35,quiet=True)

All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 447, 451, 455, 469]
{'Learners': 1, 'Environments': 170, 'Interactions': 11016}


In [ ]:
import coba as cb

class EMT:

    def __init__(self, split:int = 100, scorer:str="self_consistent_rank", bound:int=0, features=[1,'x']) -> None:

        self.params = {'split':split, 'scorer':scorer, 'bound':bound, 'features':features}

        feat_args = []
        if 1   not in features: feat_args.append('--noconstant')
        if 'a' not in features: feat_args.append('--ignore_linear a')
        if 'x' not in features: feat_args.append('--ignore_linear x')
        feat_args += [ f'--interactions {f}' for f in features if f not in {1, 'a', 'x'} ]

        vw_args = [
            "--emt",
            f"--emt_tree {bound}",
            f"--emt_leaf {split}",
            f"--emt_scorer {scorer}",
            f"--emt_router {'eigen'}",
            f"-b {26}",
            "--min_prediction 0",
            "--max_prediction 3",
            "--coin",
            "--initial_weight 0",
            *feat_args,
            '--quiet',
            '--random_seed 1337'
        ]

        self._vw = cb.VowpalMediator()
        self._vw_args = ' '.join(vw_args)

    def predict(self, X):
        if not self._vw.is_initialized: self._vw.init_learner(self._vw_args, label_type=2)
        return torch.tensor([[float(self._vw.predict(self._vw.make_example({"x":x}, None)))] for x in X.tolist()])

    def learn(self, X, Y):
        if not self._vw.is_initialized: self._vw.init_learner(self._vw_args, label_type=2)
        for x,y in zip(X.tolist(),Y.squeeze(1).tolist()):
            self._vw.learn(self._vw.make_example({'x':x}, f"{int(y)} 1"))

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr,i):
        from itertools import compress, repeat, chain
        from operator import eq

        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.train_X)))

        X = self.train_X.tolist()    
        Y = self.train_Y[:,i]
        Y = list(map(tuple,Y.tolist()))

        X = list(map(X.__getitem__,rng_indexes))
        Y = list(map(Y.__getitem__,rng_indexes))

        eq_class  = {y: list(compress(X,map(eq,Y,repeat(y)))) for y in set(Y)}
        ne_class  = {y: list(chain(*[v for k,v in eq_class.items() if k != y ])) for y in set(Y)}

        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        if sr < 1:
            anchor, positive, negative = [], [], []
            for i in range(int(len(X)*sr)):
                x,y = X[i],Y[i]
                anchor.append(x)
                positive.append(choose_unique(eq_class[y],x))
                negative.append(choose_n     (ne_class[y],neg))
            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

        else:
            for _ in range(sr):
                anchor, positive, negative = [], [], []
                for x,y in zip(X,Y):
                    anchor.append(x)
                    positive.append(choose_unique(eq_class[y],x))
                    negative.append(choose_n     (ne_class[y],neg))

                yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s0, s1, s2, s3, dae_steps, dae_dropn, ssl_samps, ssl_neg, ssl_dropn, ssl_tau, ws_steps, pers_lrn_cnt, pers_mem_cnt, pers_mem_rpt, pers_mem_rcl, y, emt_cnt, emt_bound, emt_feats, emt_split):

        self.s0  = s0  #dae + sl
        self.s1  = s1  #ssl + sl
        self.s2  = s2  #sl
        self.s3  = s3  #sl + pers

        self.dae_steps = dae_steps
        self.dae_dropn = dae_dropn

        self.ssl_samps = ssl_samps
        self.ssl_neg   = ssl_neg
        self.ssl_tau   = ssl_tau
        self.ssl_dropn = ssl_dropn

        self.ws_steps = ws_steps

        self.pers_lrn_cnt = pers_lrn_cnt
        self.pers_mem_cnt = pers_mem_cnt
        self.pers_mem_rpt = pers_mem_rpt
        self.pers_mem_rcl = pers_mem_rcl

        self.emt_cnt = emt_cnt
        self.emt_bound = emt_bound
        self.emt_feats = emt_feats
        self.emt_split = emt_split

        self.y = y

        self.params = {'s0': s0, 's1': s1, 's2':s2, 's3':s3, 'dae': (dae_steps,dae_dropn), 'ssl': (ssl_samps,ssl_neg,ssl_dropn,ssl_tau), 'ws': ws_steps, 'pers': (pers_lrn_cnt,pers_mem_cnt,pers_mem_rpt,pers_mem_rcl), 'y': y, 'emt':(emt_cnt,emt_bound,emt_feats,emt_split)}

    def evaluate(self, env, lrn):
        from sklearn.metrics import roc_auc_score

        torch.set_num_threads(1)
        torch.manual_seed(1)

        emts = [EMT(bound=self.emt_bound,features=self.emt_feats,split=self.emt_split) for _ in range(self.emt_cnt)]

        s0 = FeedForward(self.s0)
        s1 = FeedForward(self.s1)
        s2 = FeedForward(self.s2)
        s3 = FeedForward(self.s3)

        sa = torch.nn.Sequential(*list(s0.children())[len(self.s0)-self.dae_dropn:])
        s0 = torch.nn.Sequential(*list(s0.children())[:len(self.s0)-self.dae_dropn])

        sb = torch.nn.Sequential(*list(s1.children())[len(self.s1)-self.ssl_dropn:])
        s1 = torch.nn.Sequential(*list(s1.children())[:len(self.s1)-self.ssl_dropn])

        s0.train()
        sa.train()
        s1.train()
        sb.train()
        s2.train()
        s3.train()

        s0opt = COCOB(s0.parameters()) if list(s0.parameters()) else None
        saopt = COCOB(sa.parameters()) if list(sa.parameters()) else None
        s1opt = COCOB(s1.parameters()) if list(s1.parameters()) else None
        sbopt = COCOB(sb.parameters()) if list(sb.parameters()) else None
        s2opt = COCOB(s2.parameters()) if list(s2.parameters()) else None
        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        if self.dae_steps:

            torch.manual_seed(1)
            X = env.train()[0]
            torch_dataset = torch.utils.data.TensorDataset(X)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.L1Loss(reduction='none')
            for _ in range(self.dae_steps):
                for (_X,) in torch_loader:
                    if s0opt: s0opt.zero_grad()
                    if saopt: saopt.zero_grad()
                    loss(sa(s0(_X.nan_to_num())),_X).nanmean().backward()
                    if s0opt: s0opt.step()
                    if saopt: saopt.step()

        if self.ssl_samps:

            if self.ssl_neg == 0: raise Exception("neg can't be 0")
            if self.ssl_tau == 0: raise Exception("Tau can't be 0")

            torch.manual_seed(1)

            for A, P, N in env.ssl(self.ssl_neg,self.ssl_samps,self.y):

                torch_dataset = torch.utils.data.TensorDataset(A,P,N)
                torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=64,drop_last=True,shuffle=True)

                for _A, _P, _N in torch_loader:
                    #https://arxiv.org/pdf/2002.05709
                    _A = sb(s1(s0(_A.nan_to_num())))
                    _P = sb(s1(s0(_P.nan_to_num())))
                    _N = sb(s1(s0(_N.nan_to_num())))

                    p = torch.einsum("bi,bi->b",_A,_P)
                    n = torch.einsum("bi,bji->bj",_A,_N)

                    p /= (torch.linalg.norm(_A,dim=1)*torch.linalg.norm(_P,dim=1))
                    n /= (torch.linalg.norm(_A,dim=1).unsqueeze(1)*torch.linalg.norm(_N,dim=2))

                    p = torch.exp(p/self.ssl_tau)
                    n = torch.exp(n/self.ssl_tau)

                    if s1opt: s1opt.zero_grad()
                    if sbopt: sbopt.zero_grad()
                    (-torch.log(p/(p+n.sum(dim=1)))).mean().backward()
                    if s1opt: s1opt.step()
                    if sbopt: sbopt.step()

        if self.ws_steps:

            if s0opt: s0opt.zero_grad()
            if s1opt: s1opt.zero_grad()
            if s2opt: s2opt.zero_grad()
            if s3opt: s3opt.zero_grad()

            torch.manual_seed(1)
            
            X, Y = env.train()
            Y = Y[:,self.y]
            
            torch_dataset = torch.utils.data.TensorDataset(X,Y)
            torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

            loss = torch.nn.BCEWithLogitsLoss()
            for _ in range(self.ws_steps):
                for _X,_y in torch_loader:

                    if s0opt: s0opt.zero_grad()
                    if s1opt: s1opt.zero_grad()
                    if s2opt: s2opt.zero_grad()
                    if s3opt: s3opt.zero_grad()
                    loss(s3(s2(s1(s0(_X.nan_to_num())))),_y).backward()
                    if s3opt: s3opt.step()
                    if s2opt: s2opt.step()
                    if s1opt: s1opt.step()
                    if s0opt: s0opt.step()

            rng = cb.CobaRandom(1)
            for emt in emts:            
                ind = rng.shuffle(range(len(Y)))
                emt.learn(s2(s1(s0(X[ind,:].nan_to_num()))),Y[ind,:])

        s0.eval()
        s1.eval()
        s2.eval()
        s3.eval()

        torch.manual_seed(1)

        X, Y = env.test()
        Y = Y[:,self.y]

        rng = cb.CobaRandom(1)
        while 1 in [len(set(Y[35:,i].tolist())) for i in range(Y.shape[1])]:
            ind = rng.shuffle(range(len(Y)))
            X = X[ind,:]
            Y = Y[ind,:]

        s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

        lrnx = []
        lrny = []
        mems = []

        def predict(X):
            return sum([emt.predict(s2(s1(s0(X.nan_to_num())))) for emt in emts])/len(emts)

        def score(X,Y):
            with torch.no_grad():
                return {f'auc{y}': roc_auc_score(Y[:,i],predict(X[:,:])[:,i]) for i,y in enumerate(self.y) }

        yield score(X[35:,:], Y[35:,:])

        loss = torch.nn.BCEWithLogitsLoss()
        for i in range(35):

            x,y = X[[i],:], Y[[i],:]

            for emt in emts:
                emt.learn(s2(s1(s0(x.nan_to_num()))),y)

            yield score(X[35:,:], Y[35:,:])

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(1):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if 1 in [len(set(tst_y[:,i].tolist())) for i in range(tst_y.shape[1])]:
                all_equal.add(g)
                continue

            envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))

    if len(all_equal) > 0:
        print(f"All Equal, no environment added for {sorted(all_equal)}")

    return envs

lrns = [ FeedForward([]) ]
vals = lambda x: [
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 1, 75,['x'],100),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 2, 75,['x'],100),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 4, 75,['x'],100),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 8, 75,['x'],100),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 4, 100,['x'],100),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 4, 100,['x','xx'],100),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 16, 50,['x'],100),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 16, 200,['x'],100),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 16, 400,['x'],100),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 16, 800,['x'],100),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 32, 800,['x'],100),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 16, 2400,['x'],100),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 16, 800,['x'],50),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 16, 800,['x'],150),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 16, 800,['x'],200),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 16, 10000,['x'],50),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 16, 10000,['x'],100),
]
envs = make_envs(X3,np.hstack([Y1,Y2]),G)

cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/5/out5_11.log',processes=35,quiet=True)

All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 447, 451, 455, 469]
{'Learners': 1, 'Environments': 226, 'Interactions': 138312}


In [ ]:
import coba as cb

class FeedForward(torch.nn.Sequential):
    """A Generic implementation of Feedforward Neural Network"""

    class SkipModule(torch.nn.Module):
        def __init__(self, layers):
            super().__init__()
            self.layers = layers
        def forward(self,X):
            return X + self.layers(X)

    def make_layer(self,curr_dim,spec):
        if isinstance(spec,float):
            return torch.nn.Dropout(spec), curr_dim
        if curr_dim is None and isinstance(spec,int):
            return None, spec
        if isinstance(spec,int):
            return torch.nn.Linear(curr_dim,spec),spec
        if spec == 'r':
            return torch.nn.ReLU(),curr_dim
        if spec == 'l':
            return torch.nn.LayerNorm(curr_dim),curr_dim
        if spec == 'b':
            return torch.nn.BatchNorm1d(curr_dim), curr_dim
        if spec == 's':
            return torch.nn.Sigmoid(),curr_dim
        if isinstance(spec,list):                
            return FeedForward.SkipModule(FeedForward([curr_dim] + spec)), curr_dim
        raise Exception("Bad Layer")

    def __init__(self, specs, rng=1):
        """Instantiate a Feedfoward network according to specifications.

        Args:
            specs: A sequence of layer specifications as follows:
                -1 -- replaced with the input feature width
                <int> -- a LinearLayer with output width equal to <int>
                [0,1] -- a Dropout layer with the given probability
                'l' -- a LayerNorm
                'b' -- a BatchNorm1d
                'r' -- a ReLU layer
                's' -- a Sigmoid layer
                [] -- a skip layer with the given specifications
        """

        torch.manual_seed(rng)
        layers,width = [],None
        for spec in specs:
            layer,width = self.make_layer(width,spec)
            if layer: layers.append(layer)
        super().__init__(*(layers or [torch.nn.Identity()]))
        self.params = {"specs": specs, "rng": rng }

class MyEnvironment:
    def __init__(self, train_X, train_Y, test_X, test_Y, g, rng):
        self.params = {'pid': g, 'rng': rng}
        self.train_X = train_X
        self.train_Y = train_Y.float()
        self.test_X = test_X
        self.test_Y = test_Y.float()

    def ssl(self,neg,sr,i):
        from itertools import compress, repeat, chain
        from operator import eq

        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.train_X)))

        X = self.train_X.tolist()    
        Y = self.train_Y[:,i]
        Y = list(map(tuple,Y.tolist()))

        X = list(map(X.__getitem__,rng_indexes))
        Y = list(map(Y.__getitem__,rng_indexes))

        eq_class  = {y: list(compress(X,map(eq,Y,repeat(y)))) for y in set(Y)}
        ne_class  = {y: list(chain(*[v for k,v in eq_class.items() if k != y ])) for y in set(Y)}

        rng = cb.CobaRandom(self.params['rng'])

        def choose_unique(items,item):
            while items[i:=rng.randint(0,len(items)-1)]==item:
                pass
            return items[i]

        def choose_n(items,n):
            indexes = set()
            while True:
                indexes.add(rng.randint(0,len(items)-1))
                if len(indexes)==n: return [items[i] for i in indexes]

        if sr < 1:
            anchor, positive, negative = [], [], []
            for i in range(int(len(X)*sr)):
                x,y = X[i],Y[i]
                anchor.append(x)
                positive.append(choose_unique(eq_class[y],x))
                negative.append(choose_n     (ne_class[y],neg))
            yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

        else:
            for _ in range(sr):
                anchor, positive, negative = [], [], []
                for x,y in zip(X,Y):
                    anchor.append(x)
                    positive.append(choose_unique(eq_class[y],x))
                    negative.append(choose_n     (ne_class[y],neg))

                yield torch.tensor(anchor).float(), torch.tensor(positive).float(), torch.tensor(negative).float()

    def train(self):
        return self.train_X, self.train_Y

    def test(self):
        rng_indexes = cb.CobaRandom(self.params['rng']).shuffle(range(len(self.test_X)))
        return self.test_X[rng_indexes,:], self.test_Y[rng_indexes]

class MyEvaluator:
    def __init__(self, s0, s1, s2, s3, dae_steps, dae_dropn, ssl_samps, ssl_neg, ssl_dropn, ssl_tau, ws_steps, pers_lrn_cnt, pers_mem_cnt, pers_mem_rpt, pers_mem_rcl, y, n_models):

        self.s0  = s0  #dae + sl
        self.s1  = s1  #ssl + sl
        self.s2  = s2  #sl
        self.s3  = s3  #sl + pers

        self.dae_steps = dae_steps
        self.dae_dropn = dae_dropn

        self.ssl_samps = ssl_samps
        self.ssl_neg   = ssl_neg
        self.ssl_tau   = ssl_tau
        self.ssl_dropn = ssl_dropn

        self.ws_steps = ws_steps

        self.pers_lrn_cnt = pers_lrn_cnt
        self.pers_mem_cnt = pers_mem_cnt
        self.pers_mem_rpt = pers_mem_rpt
        self.pers_mem_rcl = pers_mem_rcl

        self.n_models = n_models

        self.y = y

        self.params = {'s0': s0, 's1': s1, 's2':s2, 's3':s3, 'dae': (dae_steps,dae_dropn), 'ssl': (ssl_samps,ssl_neg,ssl_dropn,ssl_tau), 'ws': ws_steps, 'pers': (pers_lrn_cnt,pers_mem_cnt,pers_mem_rpt,pers_mem_rcl), 'y': y, 'n_models': n_models}

    def evaluate(self, env, lrn):
        from sklearn.metrics import roc_auc_score

        torch.set_num_threads(1)
        torch.manual_seed(1)

        mods_opts = []
        opts = []

        for _ in range(self.n_models):
            s0 = FeedForward(self.s0)
            s1 = FeedForward(self.s1)
            s2 = FeedForward(self.s2)
            s3 = FeedForward(self.s3)
            sa = torch.nn.Sequential(*list(s0.children())[len(self.s0)-self.dae_dropn:])
            s0 = torch.nn.Sequential(*list(s0.children())[:len(self.s0)-self.dae_dropn])
            sb = torch.nn.Sequential(*list(s1.children())[len(self.s1)-self.ssl_dropn:])
            s1 = torch.nn.Sequential(*list(s1.children())[:len(self.s1)-self.ssl_dropn])

            s0opt = COCOB(s0.parameters()) if list(s0.parameters()) else None
            saopt = COCOB(sa.parameters()) if list(sa.parameters()) else None
            s1opt = COCOB(s1.parameters()) if list(s1.parameters()) else None
            sbopt = COCOB(sb.parameters()) if list(sb.parameters()) else None
            s2opt = COCOB(s2.parameters()) if list(s2.parameters()) else None
            s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None

            mods = [s0,sa,s1,sb,s2,s3]
            opts = [s0opt,saopt,s1opt,sbopt,s2opt,s3opt]
            mods_opts.append([mods,opts])

        for mods,_ in mods_opts:
            for l in mods: l.train()

        for mods,opts in mods_opts:
            [s0,sa,s1,sb,s2,s3] = mods
            [s0opt,saopt,s1opt,sbopt,s2opt,s3opt] = opts

            if self.dae_steps:

                X = env.train()[0]
                #torch.manual_seed(1)
                torch_dataset = torch.utils.data.TensorDataset(X)
                torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)
                loss = torch.nn.L1Loss(reduction='none')

                for _ in range(self.dae_steps):
                    for (_X,) in torch_loader:
                        if s0opt: s0opt.zero_grad()
                        if saopt: saopt.zero_grad()
                        loss(sa(s0(_X.nan_to_num())),_X).nanmean().backward()
                        if s0opt: s0opt.step()
                        if saopt: saopt.step()

            if self.ssl_samps:

                if self.ssl_neg == 0: raise Exception("neg can't be 0")
                if self.ssl_tau == 0: raise Exception("Tau can't be 0")
                #torch.manual_seed(1)
                for A, P, N in env.ssl(self.ssl_neg,self.ssl_samps,self.y):

                    torch_dataset = torch.utils.data.TensorDataset(A,P,N)
                    torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=64,drop_last=True,shuffle=True)

                    for _A, _P, _N in torch_loader:
                        #https://arxiv.org/pdf/2002.05709
                        _A = sb(s1(s0(_A.nan_to_num())))
                        _P = sb(s1(s0(_P.nan_to_num())))
                        _N = sb(s1(s0(_N.nan_to_num())))

                        p = torch.einsum("bi,bi->b",_A,_P)
                        n = torch.einsum("bi,bji->bj",_A,_N)

                        p /= (torch.linalg.norm(_A,dim=1)*torch.linalg.norm(_P,dim=1))
                        n /= (torch.linalg.norm(_A,dim=1).unsqueeze(1)*torch.linalg.norm(_N,dim=2))

                        p = torch.exp(p/self.ssl_tau)
                        n = torch.exp(n/self.ssl_tau)

                        if s1opt: s1opt.zero_grad()
                        if sbopt: sbopt.zero_grad()
                        (-torch.log(p/(p+n.sum(dim=1)))).mean().backward()
                        if s1opt: s1opt.step()
                        if sbopt: sbopt.step()

            if self.ws_steps:

                if s0opt: s0opt.zero_grad()
                if s1opt: s1opt.zero_grad()
                if s2opt: s2opt.zero_grad()
                if s3opt: s3opt.zero_grad()

                #torch.manual_seed(1)

                X, Y = env.train()
                Y = Y[:,self.y]

                torch_dataset = torch.utils.data.TensorDataset(X,Y)
                torch_loader  = torch.utils.data.DataLoader(torch_dataset,batch_size=16,drop_last=True,shuffle=True)

                loss = torch.nn.BCEWithLogitsLoss()
                for _ in range(self.ws_steps):
                    for _X,_y in torch_loader:

                        if s0opt: s0opt.zero_grad()
                        if s1opt: s1opt.zero_grad()
                        if s2opt: s2opt.zero_grad()
                        if s3opt: s3opt.zero_grad()
                        loss(s3(s2(s1(s0(_X.nan_to_num())))),_y).backward()
                        if s3opt: s3opt.step()
                        if s2opt: s2opt.step()
                        if s1opt: s1opt.step()
                        if s0opt: s0opt.step()

        for mods,_ in mods_opts:
            for l in mods: l.eval()

        X, Y = env.test()
        Y = Y[:,self.y]

        rng = cb.CobaRandom(1)
        while 1 in [len(set(Y[35:,i].tolist())) for i in range(Y.shape[1])]:
            ind = rng.shuffle(range(len(Y)))
            X = X[ind,:]
            Y = Y[ind,:]

        for mods,opts in mods_opts:
            s3 = mods[5]
            s3opt = COCOB(s3.parameters()) if list(s3.parameters()) else None
            opts[5] = s3opt

        lrnxs = [[] for _ in range(len(mods_opts))]
        lrnys = [[] for _ in range(len(mods_opts))]
        memss = [[] for _ in range(len(mods_opts))]

        def predict(X):
            preds = []
            for mods, _ in mods_opts:
                [s0,_,s1,_,s2,s3] = mods
                preds.append(torch.sigmoid(s3(s2(s1(s0(X.nan_to_num()))))))
            return sum(preds)/len(preds)

        def score(X,Y):
            with torch.no_grad():
                return {f'auc{y}': roc_auc_score(Y[:,i],predict(X[:,:])[:,i]) for i,y in enumerate(self.y) }

        yield score(X[35:,:], Y[35:,:])

        loss = torch.nn.BCEWithLogitsLoss()
        for i in range(35):

            for lrnx,lrny,mems,(mods,opts) in zip(lrnxs,lrnys,memss,mods_opts):
                [s0,_,s1,_,s2,s3] = mods
                s3opt = opts[5]

                x,y = X[i,:], Y[i,:]

                if self.pers_lrn_cnt:
                    lrnx.append(x)
                    lrny.append(y)

                if self.pers_mem_cnt: 
                    mems.append([x,y,self.pers_mem_rpt])

                if len(mems) > self.pers_mem_cnt:
                    rng.shuffle(mems, inplace=True)
                    for j in reversed(range(self.pers_mem_rcl)):
                        x,y,n = mems[j]
                        lrnx.append(x)
                        lrny.append(y)
                        if n == 1: mems.pop(j)
                        else: mems[j] = [x,y,n-1]

                if len(lrnx) >= self.pers_lrn_cnt:
                    x = torch.stack(lrnx[:self.pers_lrn_cnt])
                    y = torch.stack(lrny[:self.pers_lrn_cnt])
                    if s3opt: s3opt.zero_grad()
                    loss(s3(s2(s1(s0(x.nan_to_num())))),y).backward()
                    if s3opt: s3opt.step()
                    del lrnx[:self.pers_lrn_cnt]
                    del lrny[:self.pers_lrn_cnt]

            yield score(X[35:,:], Y[35:,:])

def make_envs(X, Y, G):
    X, Y, G = torch.tensor(X).float(), torch.tensor(Y).float(), torch.tensor(G)
    all_equal = set()
    envs = []
    for rng in range(5):
        for g in sorted(set(G.tolist())):
            if (g==G).sum() < 50: continue
            trn_x,trn_y = X[g!=G],Y[g!=G]
            tst_x,tst_y = X[g==G],Y[g==G]

            if 1 in [len(set(tst_y[:,i].tolist())) for i in range(tst_y.shape[1])]:
                all_equal.add(g)
                continue

            envs.append(MyEnvironment(trn_x,trn_y,tst_x,tst_y,g,rng))

    if len(all_equal) > 0:
        print(f"All Equal, no environment added for {sorted(all_equal)}")

    return envs

lrns = [ FeedForward([]) ]
vals = lambda x: [
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 1),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 2),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 4),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 8),
    MyEvaluator((len(x),.2,90 ,'l','r',len(x)), (), (90,30,'l','r',), (30,1), 2, 3, 0, 0, 0, 0, 3, 3, 20, 2, 2, [1], 16),
]

envs = make_envs(X3,np.hstack([Y1,Y2]),G)
cb.Experiment(envs,lrns,vals(X3[0])).run('../logs/5/out5_12.log',processes=35,quiet=True)

All Equal, no environment added for [2, 25, 61, 82, 87, 89, 99, 103, 111, 117, 146, 196, 203, 227, 246, 311, 319, 357, 377, 390, 447, 451, 455, 469]
{'Learners': 1, 'Environments': 1130, 'Interactions': 203400}
